In [87]:
import os
import pandas as pd
import numpy as np
from datetime import timedelta, datetime
import datetime as dt
import copy

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

#### Chuẩn bị dữ liệu

##### Các dữ liệu dùng để làm map tham chiếu

In [88]:
#Đọc name map để chuyển đỏi các tên thành dạng full
name_map = pd.read_excel("../period_data/stock_classification.xlsx", sheet_name='name_map').drop(columns=['group', 'order'],axis=1)
name_map_dict = name_map.set_index('code')['full_name'].to_dict()

order_map = pd.read_excel("../period_data/stock_classification.xlsx", sheet_name='name_map').drop(columns=['group', 'full_name'],axis=1)
order_map_dict = order_map.set_index('code')['order'].to_dict()

group_map = pd.read_excel("../period_data/stock_classification.xlsx", sheet_name='name_map').drop(columns=['order', 'full_name'],axis=1)
group_map_dict = group_map.set_index('code')['group'].to_dict()

#Tạo các danh sách nhóm trong mỗi cách chia cổ phiếu
all_stock_key_list = [key for key, value in group_map_dict.items() if value == 'tt']
industry_name_list = [key for key, value in group_map_dict.items() if value in ['A', 'B', 'C', 'D']]
industry_perform_list = [key for key, value in group_map_dict.items() if value == 'hs']
marketcap_group_list = [key for key, value in group_map_dict.items() if value == 'cap']

#Tạo danh danh key cho tổng tất cả các nhóm
group_stock_key_list = all_stock_key_list + industry_name_list + industry_perform_list + marketcap_group_list

In [89]:
#Tạo dict map thời gian và số lượng cổ phiếu
period_map = pd.read_excel("../period_data/period_stock_list.xlsx", sheet_name='period_map')
period_map_dict = period_map.set_index('index').apply(lambda row: row.tolist(), axis=1).to_dict()

#Xoá đi quý hiện tại để chỉ tính toán tới quý trước đó
def get_quarter(name):
    now = datetime.now()
    year = now.year
    month = now.month
    if 1 <= month <= 3:
        quarter = "q1"
        previous_quarter = "q4"
    elif 4 <= month <= 6:
        quarter = "q2"
        previous_quarter = "q3"
    elif 7 <= month <= 9:
        quarter = "q3"
        previous_quarter = "q2"
    else:
        quarter = "q4"
        previous_quarter = "q1"
    if name == 'current_quarter':
        return f'{quarter}_{year}'
    if name == 'previous_quarter':
        if quarter == 'q4':
            return f'{previous_quarter}_{year-1}'
        else:
            return f'{previous_quarter}_{year}'
        
#Lấy ra khoảng thời gian tính toán cho quý này và quý trước
calculate_time_span = [period_map_dict['q2_2020'][0], period_map_dict[get_quarter('current_quarter')][1]]
current_quarter_span = [period_map_dict[get_quarter('current_quarter')][0], period_map_dict[get_quarter('current_quarter')][1]]
previous_quarter_span = [period_map_dict[get_quarter('previous_quarter')][0], period_map_dict[get_quarter('previous_quarter')][1]]

#Lấy ra list cổ phiếu của giai đoạn hiện tại
period_stock_list = pd.read_excel("../period_data/period_stock_list.xlsx", sheet_name='period_stock_list')
curren_stock_list = period_stock_list[get_quarter('current_quarter')].dropna().tolist()

##### Các biến thời gian

In [90]:
#Khởi tạo ngày và thời gian hiện tại
today = pd.to_datetime(
        pd.read_csv('D:\\t2m-project\\ami-data\\ami_eod_data\\VNINDEX.csv')\
        .sort_values('date', ascending=False).reset_index(drop=True)\
        ['date'].iloc[0]
        , format='%y%m%d')

current_time = pd.to_datetime(
               pd.read_csv('D:\\t2m-project\\ami-data\\ami_itd_data\\HNXINDEX.csv')\
               .sort_values('date', ascending=False).reset_index(drop=True)\
               ['date'].iloc[0]
               , format='%y%m%d %H%M%S')

#Khởi tạo time_series bao gồm tất cả khung thời gian của ngày hiện tại
time_series_list = []
time_series_list.extend(pd.date_range(start=f'{today} 09:00:00', end=f'{today} 11:25:00', freq='5T'))
time_series_list.extend(pd.date_range(start=f'{today} 13:00:00', end=f'{today} 14:55:00', freq='5T'))
time_series = pd.DataFrame(time_series_list).rename(columns={0:'date'})

#Khởi tạo khung thời gian bắt đầu từ 9h15 để vẽ các biểu đồ
itd_series = pd.DataFrame(time_series_list[3:]).rename(columns={0:'date'})

In [91]:
def calculate_time_percent(time):
    start_time_am = dt.time(9, 00)
    end_time_am = dt.time(11, 30)
    start_time_pm = dt.time(13, 00)
    end_time_pm = dt.time(15, 00)

    def time_difference_in_minutes(time1, time2):
        delta1 = dt.timedelta(hours=time1.hour, minutes=time1.minute, seconds=time1.second)
        delta2 = dt.timedelta(hours=time2.hour, minutes=time2.minute, seconds=time2.second)
        diff = delta2 - delta1
        return diff.seconds // 60

    time = (time + timedelta(minutes=5)).time()
    full_time_range = time_difference_in_minutes(start_time_am, end_time_am) + time_difference_in_minutes(start_time_pm, end_time_pm)

    if time <= end_time_am:
        time_range = time_difference_in_minutes(start_time_am, time)
    elif time >= start_time_pm:
        time_range = time_difference_in_minutes(start_time_am, time) - time_difference_in_minutes(end_time_am, start_time_pm)

    return time_range/full_time_range

#Điều chỉnh lại time_series bỏ đi các hàng thời gian chưa có dữ liệu
time_series = time_series.loc[time_series['date'] <= current_time].sort_values('date', ascending=False).reset_index(drop=True)

#Tính thêm time percent
time_percent = time_series.copy()
time_percent['percent'] = time_percent['date'].apply(calculate_time_percent)
time_percent['percent'] = time_percent['percent'].apply(lambda x: x if x < 1 else 1)

if len(time_percent['percent']) >= 1:
    current_time_percent = time_percent['percent'].iloc[0]
else: 
    current_time_percent = 1

In [92]:
#Tạo bảng thời gian update
def get_update_time(start_time_am, end_time_am, start_time_pm, end_time_pm):
    if (dt.datetime.now()).weekday() <= 4:
        current_time = dt.datetime.now().time()
        if current_time < start_time_am: current_time = end_time_pm
        elif (current_time >= start_time_am) & (current_time < end_time_am): current_time = current_time
        elif (current_time >= end_time_am) & (current_time < start_time_pm): current_time = end_time_am
        elif (current_time >= start_time_pm) & (current_time < end_time_pm): current_time = current_time
        elif current_time >= end_time_pm: current_time = end_time_pm
        return current_time
    if (dt.datetime.now()).weekday() > 4:
        return end_time_pm

time_update = get_update_time(dt.time(9, 00), dt.time(11, 30), dt.time(13, 00), dt.time(15, 00))
date_time_update = dt.datetime.combine(current_time.date(), time_update)
update_time = pd.DataFrame([f"Cập nhât: {date_time_update.strftime('%d/%m/%Y %H:%M:%S')}"]).rename(columns={0:'date'})

##### Các dữ liệu về cổ phiếu và index

In [93]:
#Đọc toàn bộ các file csv được xuất ra từ ami eod
eod_item_dict = {}
folder_path = 'D:\\t2m-project\\ami-data\\ami_eod_data'
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        key = os.path.splitext(filename)[0]
        eod_item_dict[key] = pd.read_csv(os.path.join(folder_path, filename)).sort_values('date', ascending=False).reset_index(drop=True)

for item, df in eod_item_dict.items():
    df['date'] = pd.to_datetime(df['date'], format='%y%m%d')
    eod_item_dict[item] = df

#Tạo bảng tổng hớp tất cả các item
eod_item_df = pd.DataFrame(list(eod_item_dict.keys())).rename(columns={0:'item'})
eod_item_df['len'] = eod_item_df['item'].apply(lambda x: len(x))
eod_item_df['last_2chars'] = eod_item_df['item'].str[-2:]
eod_item_df['first_4chars'] = eod_item_df['item'].str[:4]

#Lọc ra danh sách tên các cổ phiếu và index
index_name_df = eod_item_df[(eod_item_df['len']>3) & (eod_item_df['len']!=6) & (eod_item_df['len']<10) & 
                (eod_item_df['item']!='0001')].reset_index(drop=True).drop(['len','last_2chars','first_4chars'], axis=1)
                
full_stock_dict = {k:v.drop(['option'], axis=1) for k,v in eod_item_dict.items() if len(k) == 3}
eod_stock_dict = {k:v.drop(['option'], axis=1) for k,v in eod_item_dict.items()  if k in curren_stock_list}
eod_index_dict = {k:v.rename(columns={'option':'value'}).drop('cap', axis=1)
                for k,v in eod_item_dict.items() if k in index_name_df['item'].tolist()}

- Tính toán cho full_stock_dict để phục vụ cho tính điểm dòng tiền lịch sử

In [94]:
#Tính toán các đường trung bình và các đường MA cho full_stock_dict
full_stock_dict = {k: v.sort_values(by=['date'], ascending=True).reset_index(drop=True) for k, v in full_stock_dict.items()}

full_stock_dict = {
    key: df.assign(
        high5=df['high'].rolling(window=5, min_periods=1).max(),
        low5=df['low'].rolling(window=5, min_periods=1).min(),
        high20=df['high'].rolling(window=20, min_periods=1).max(),
        low20=df['low'].rolling(window=20, min_periods=1).min(),
        high60=df['high'].rolling(window=60, min_periods=1).max(),
        low60=df['low'].rolling(window=60, min_periods=1).min(),
        high120=df['high'].rolling(window=120, min_periods=1).max(),
        low120=df['low'].rolling(window=120, min_periods=1).min(),
        high240=df['high'].rolling(window=240, min_periods=1).max(),
        low240=df['low'].rolling(window=240, min_periods=1).min(),
        high480=df['high'].rolling(window=480, min_periods=1).max(),
        low480=df['low'].rolling(window=480, min_periods=1).min(),

        ma5_V=df['volume'].rolling(window=5, min_periods=1).mean().shift(1),

        ma5=df['close'].rolling(window=5, min_periods=1).mean(),
        ma20=df['close'].rolling(window=20, min_periods=1).mean(),
        ma60=df['close'].rolling(window=60, min_periods=1).mean(),
        ma120=df['close'].rolling(window=120, min_periods=1).mean(),
        ma240=df['close'].rolling(window=240, min_periods=1).mean(),
        ma480=df['close'].rolling(window=480, min_periods=1).mean(),
    )
    for key, df in full_stock_dict.items()
}

full_stock_dict = {
    key: df.assign(
        trend_5p=(df['close'] > ((df['high5'] + df['low5'])/2).shift(1)).astype(int),
        trend_20p=(df['close'] > ((df['high20'] + df['low20'])/2).shift(1)).astype(int),
        trend_60p=(df['close'] > ((df['high60'] + df['low60'])/2).shift(1)).astype(int),
        trend_120p=(df['close'] > ((df['high120'] + df['low120'])/2).shift(1)).astype(int),
        trend_240p=(df['close'] > ((df['high240'] + df['low240'])/2).shift(1)).astype(int),
        trend_480p=(df['close'] > ((df['high480'] + df['low480'])/2).shift(1)).astype(int)
    )
    for key, df in full_stock_dict.items()
}
full_stock_dict = {k: v[(v['date'] >= calculate_time_span[0]) & (v['date'] <= calculate_time_span[1])].sort_values(by=['date'], ascending=False).reset_index(drop=True) for k, v in full_stock_dict.items()}

In [95]:
#Thêm cột tên period và số lượng cổ phiếu từng thời kì
def assign_period(x):
    for key, value in period_map_dict.items():
        if (x >= pd.Timestamp(value[0])) & (x <= pd.Timestamp(value[1])):
            return key

for stock, df in full_stock_dict.items():
    df['period'] = df['date'].apply(assign_period)
    df['count'] = df['period'].apply(lambda x: period_map_dict[x][2])

#Tính hệ số thanh khoản và đổi lại cap của cổ phiếu thành cap trung bình trong 20 phiên
for df in full_stock_dict.values():
    df['liquid_ratio'] = df['volume'] / (df['ma5_V'])
    df['liquid_ratio'].iloc[0] = df['volume'].iloc[0] / ((df['ma5_V']).iloc[0]*current_time_percent)
    df['cap'] = df['cap'][::-1].rolling(window=20).mean()[::-1]

- Tính toán cho eod_stock_dict dể phục vụ phân nhóm cho quý hiện tại

In [96]:
#Tính toán các đường trung bình và các đường MA cho eod_stock_dict
eod_stock_dict = {k: v.sort_values(by=['date'], ascending=True).reset_index(drop=True) for k, v in eod_stock_dict.items()}

eod_stock_dict = {
    key: df.assign(
        high5=df['high'].rolling(window=5, min_periods=1).max(),
        low5=df['low'].rolling(window=5, min_periods=1).min(),
        high20=df['high'].rolling(window=20, min_periods=1).max(),
        low20=df['low'].rolling(window=20, min_periods=1).min(),
        high60=df['high'].rolling(window=60, min_periods=1).max(),
        low60=df['low'].rolling(window=60, min_periods=1).min(),
        high120=df['high'].rolling(window=120, min_periods=1).max(),
        low120=df['low'].rolling(window=120, min_periods=1).min(),
        high240=df['high'].rolling(window=240, min_periods=1).max(),
        low240=df['low'].rolling(window=240, min_periods=1).min(),
        high480=df['high'].rolling(window=480, min_periods=1).max(),
        low480=df['low'].rolling(window=480, min_periods=1).min(),

        ma5_V=df['volume'].rolling(window=5, min_periods=1).mean().shift(1),

        ma5=df['close'].rolling(window=5, min_periods=1).mean(),
        ma20=df['close'].rolling(window=20, min_periods=1).mean(),
        ma60=df['close'].rolling(window=60, min_periods=1).mean(),
        ma120=df['close'].rolling(window=120, min_periods=1).mean(),
        ma240=df['close'].rolling(window=240, min_periods=1).mean(),
        ma480=df['close'].rolling(window=480, min_periods=1).mean(),
    )
    for key, df in eod_stock_dict.items()
}

eod_stock_dict = {
    key: df.assign(
        trend_5p=(df['close'] > ((df['high5'] + df['low5'])/2).shift(1)).astype(int),
        trend_20p=(df['close'] > ((df['high20'] + df['low20'])/2).shift(1)).astype(int),
        trend_60p=(df['close'] > ((df['high60'] + df['low60'])/2).shift(1)).astype(int),
        trend_120p=(df['close'] > ((df['high120'] + df['low120'])/2).shift(1)).astype(int),
        trend_240p=(df['close'] > ((df['high240'] + df['low240'])/2).shift(1)).astype(int),
        trend_480p=(df['close'] > ((df['high480'] + df['low480'])/2).shift(1)).astype(int)
    )
    for key, df in eod_stock_dict.items()
}
eod_stock_dict = {k: v[(v['date'] >= calculate_time_span[0]) & (v['date'] <= calculate_time_span[1])].sort_values(by=['date'], ascending=False).reset_index(drop=True) for k, v in eod_stock_dict.items()}

In [97]:
#Thêm cột tên period và số lượng cổ phiếu từng thời kì
def assign_period(x):
    for key, value in period_map_dict.items():
        if (x >= pd.Timestamp(value[0])) & (x <= pd.Timestamp(value[1])):
            return key

for stock, df in eod_stock_dict.items():
    df['period'] = df['date'].apply(assign_period)
    df['count'] = df['period'].apply(lambda x: period_map_dict[x][2])

#Tính hệ số thanh khoản và đổi lại cap của cổ phiếu thành cap trung bình trong 20 phiên
for df in eod_stock_dict.values():
    df['liquid_ratio'] = df['volume'] / (df['ma5_V'])
    df['liquid_ratio'].iloc[0] = df['volume'].iloc[0] / ((df['ma5_V']).iloc[0]*current_time_percent)
    df['cap'] = df['cap'][::-1].rolling(window=20).mean()[::-1]

- Tạo date_series full lịch sử

In [98]:
#Tạo một date_series bao gồm khoảng ngày tính toán eod
date_series = pd.DataFrame(full_stock_dict['REE']['date']).rename(columns={0:'date'})

#### Chia nhóm cổ phiếu quý hiện tại

In [99]:
full_stock_classification_df = pd.read_excel("../period_data/stock_classification.xlsx", sheet_name='stock_classification')
stock_classification_df = full_stock_classification_df[full_stock_classification_df['stock'].isin(curren_stock_list)].reset_index(drop=True)

price_arr = []
cap_arr = []
for stock, df in eod_stock_dict.items():
    df = df[df['date'] > current_quarter_span[0]]
    price_arr.append(df['close'].iloc[-1].item())
    cap_arr.append(df['cap'].iloc[-1].item())

vonhoa_classification_df = stock_classification_df.copy()
vonhoa_classification_df['price'] = price_arr
vonhoa_classification_df['cap'] = cap_arr

cap_coef = sum(cap_arr)/10000
vonhoa_classification_df['marketcap_group'] = vonhoa_classification_df.apply(lambda x:
'small' if ((x['cap']>cap_coef) & (x['cap']<10*cap_coef)) | 
        ((x['cap']>=10*cap_coef) & (x['cap']<20*cap_coef) & (x['price']<10)) 
        else
('mid' if ((x['cap']>=10*cap_coef) & (x['cap']<20*cap_coef) & (x['price']>=10)) | 
        ((x['cap']>=20*cap_coef) & (x['cap']<100*cap_coef))
        else
('large' if x['cap']>=100*cap_coef
        else 'penny'
)), axis=1)

stock_classification_df = pd.concat([stock_classification_df, vonhoa_classification_df['marketcap_group']], axis=1)

In [100]:
# Convert DataFrame columns to dictionaries for quick access
stock_by_industry = stock_classification_df.set_index('stock')['industry_name'].to_dict()
stock_by_perform = stock_classification_df.set_index('stock')['industry_perform'].to_dict()
stock_by_marketcap = stock_classification_df.set_index('stock')['marketcap_group'].to_dict()

# Initialize dictionaries
eod_all_stock = {}
eod_industry_name = {}
eod_industry_perform = {}
eod_marketcap_group = {}

# Function to create mappings based on category
def create_mapping(stock_dict, category_dict):
    category_map = {}
    for category, stocks in category_dict.items():
        category_map[category] = {stock: stock_dict[stock] for stock in stocks if stock in stock_dict}
    return category_map

# Precompute unique categories and relevant stocks
unique_industries = np.unique(list(stock_by_industry.values()))
unique_performs = np.unique(list(stock_by_perform.values()))
unique_marketcaps = ['large', 'mid', 'small', 'penny']

# Mapping for all_stock
eod_all_stock['all_stock'] = {key: value for key, value in eod_stock_dict.items()}

# Mapping for industry
for industry in unique_industries:
    relevant_stocks = [stock for stock, ind in stock_by_industry.items() if ind == industry]
    eod_industry_name[industry] = {stock: eod_stock_dict[stock] for stock in relevant_stocks if stock in eod_stock_dict}

# Mapping for performance
for performance in unique_performs:
    relevant_stocks = [stock for stock, perf in stock_by_perform.items() if perf == performance]
    eod_industry_perform[performance] = {stock: eod_stock_dict[stock] for stock in relevant_stocks if stock in eod_stock_dict}

# Mapping for marketcap
for marketcap in unique_marketcaps:
    relevant_stocks = [stock for stock, mcap in stock_by_marketcap.items() if mcap == marketcap]
    eod_marketcap_group[marketcap] = {stock: eod_stock_dict[stock] for stock in relevant_stocks if stock in eod_stock_dict}


#### Chia nhóm cổ phiếu từng quý

In [101]:
#Bảng danh sách tất cả các cổ phiếu trong tất cả giai đoạn (không bao gồm hiện tại)
period_stock_df = pd.read_excel("../period_data/stock_classification.xlsx", sheet_name='period_stock_list')

#Dict chứa dữ liệu cổ phiếu cho từng giai đoán
period_stock_dict = {}
for period in period_stock_df.columns[1:].tolist():
    period_stock_dict[period] = {k: v[(v['date'] >= period_map_dict[period][0]) & 
                                      (v['date'] <= period_map_dict[period][1])].reset_index(drop=True)
                                      for k, v in copy.deepcopy(full_stock_dict).items()
                                      if k in period_stock_df[period].dropna().tolist()}

#Dict chưa date series cho từng giai đoạn
period_date_series_dict = {}
for period in period_stock_df.columns[1:].tolist():
    period_date_series_dict[period] = date_series[(date_series['date'] >= period_map_dict[period][0]) & 
                                                  (date_series['date'] <= period_map_dict[period][1])].reset_index(drop=True)

In [102]:
full_stock_classification_df = pd.read_excel("../period_data/stock_classification.xlsx", sheet_name='stock_classification')
full_stock_classification_df = full_stock_classification_df[full_stock_classification_df['stock'].isin(full_stock_dict.keys())]

period_stock_classification_dict = {}
for period in period_stock_df.columns[1:].tolist():
    period_stock_classification_dict[period] = full_stock_classification_df[full_stock_classification_df['stock'].isin(period_stock_df[period].dropna().tolist())].reset_index(drop=True)
    period_stock_classification_dict[period] = period_stock_classification_dict[period][period_stock_classification_dict[period]['stock'].isin(full_stock_dict.keys())]
    
    price_arr = []
    cap_arr = []
    for stock, df in period_stock_dict[period].items():
        if len(df) > 0:
            price_arr.append(df['close'].iloc[-1].item())
            cap_arr.append(df['cap'].iloc[-1].item())
        else:
            price_arr.append(0)
            cap_arr.append(0)

    vonhoa_classification_df = period_stock_classification_dict[period].copy()
    vonhoa_classification_df['price'] = price_arr
    vonhoa_classification_df['cap'] = cap_arr

    cap_coef = sum(cap_arr)/10000
    vonhoa_classification_df['marketcap_group'] = vonhoa_classification_df.apply(lambda x:
        'small' if ((x['cap']>cap_coef) & (x['cap']<10*cap_coef)) | 
                ((x['cap']>=10*cap_coef) & (x['cap']<20*cap_coef) & (x['price']<10)) 
                else
        ('mid' if ((x['cap']>=10*cap_coef) & (x['cap']<20*cap_coef) & (x['price']>=10)) | 
                ((x['cap']>=20*cap_coef) & (x['cap']<100*cap_coef))
                else
        ('large' if x['cap']>=100*cap_coef
                else 'penny'
    )), axis=1)

    period_stock_classification_dict[period] = pd.concat([period_stock_classification_dict[period], vonhoa_classification_df['marketcap_group']], axis=1)

In [103]:
period_all_stock = {}
period_industry_name = {}
period_industry_perform = {}
period_marketcap_group = {}

# Function to create mappings based on category
def create_mapping(stock_dict, category_dict):
    category_map = {}
    for category, stocks in category_dict.items():
        category_map[category] = {stock: stock_dict[stock] for stock in stocks if stock in stock_dict}
    return category_map

for period in period_stock_df.columns[1:].tolist():

    period_all_stock[period] = {}
    period_industry_name[period] = {}
    period_industry_perform[period] = {}
    period_marketcap_group[period] = {}

    stock_by_industry = period_stock_classification_dict[period].set_index('stock')['industry_name'].to_dict()
    stock_by_perform = period_stock_classification_dict[period].set_index('stock')['industry_perform'].to_dict()
    stock_by_marketcap = period_stock_classification_dict[period].set_index('stock')['marketcap_group'].to_dict()

    unique_industries = np.unique(list(stock_by_industry.values()))
    unique_performs = np.unique(list(stock_by_perform.values()))
    unique_marketcaps = ['large', 'mid', 'small', 'penny']

    # Mapping for all_stock
    period_all_stock[period]['all_stock'] = {key: value for key, value in period_stock_dict[period].items()}

    # Mapping for industry
    for industry in unique_industries:
        relevant_stocks = [stock for stock, ind in stock_by_industry.items() if ind == industry]
        period_industry_name[period][industry] = {stock: period_stock_dict[period][stock] for stock in relevant_stocks if stock in period_stock_dict[period]}

    # Mapping for performance
    for performance in unique_performs:
        relevant_stocks = [stock for stock, perf in stock_by_perform.items() if perf == performance]
        period_industry_perform[period][performance] = {stock: period_stock_dict[period][stock] for stock in relevant_stocks if stock in period_stock_dict[period]}

    # Mapping for marketcap
    for marketcap in unique_marketcaps:
        relevant_stocks = [stock for stock, mcap in stock_by_marketcap.items() if mcap == marketcap]
        period_marketcap_group[period][marketcap] = {stock: period_stock_dict[period][stock] for stock in relevant_stocks if stock in period_stock_dict[period]}

#### Điểm dòng tiền từng cổ phiếu quý hiện tại

In [104]:
def score_calculation(row):
    try:
        return (((row['close'] - row['low']) - (row['high'] - row['close'])) / (row['high'] - row['low']) *
                abs((row['close'] - row['close_prev'])) / row['close_prev'] *
                (row['volume']*row['close']) / (row['ma5_prev'] * row['ma5_V'])) * 100 \
                + ((row['close'] - row['ma5_prev']) / row['ma5_prev'])/100 #Cộng thêm lượng này để tránh các trường hợp điểm dòng tiền bằng nhau gây trùng xếp hạng
    except ZeroDivisionError:
        return ((row['close'] - row['ma5_prev']) / row['ma5_prev'])/100 #Cộng thêm lượng này để tránh các trường hợp điểm dòng tiền bằng nhau gây trùng xếp hạng


In [105]:
#Tính điểm dòng tiền cho từng cổ phiếu
eod_score_dict = {}
for stock in eod_stock_dict.keys():

    #Lọc ra các cột cần sử dụng và chỉ lấy 40 phiên gần nhất để tính
    temp_df = eod_stock_dict[stock][['stock', 'date', 'period', 'count', 'open', 'high', 'low', 'close', 'volume', 'liquid_ratio', 'ma5', 'ma5_V']]

    #Tính điểm dòng tiền t0 và t5
    temp_df['ma5_prev'] = temp_df['ma5'].shift(-1)
    temp_df['close_prev'] = temp_df['close'].shift(-1)
    temp_df['t0_score'] = temp_df.apply(score_calculation, axis=1)
    temp_df['t5_score'] = temp_df['t0_score'][::-1].rolling(window=5, min_periods=1).mean()[::-1]

    #Gán lại temp_df cho dict
    eod_score_dict[stock] = temp_df

#### Điểm dòng tiền từng cổ phiếu full lịch sử

In [106]:
def score_calculation(row):
    try:
        return (((row['close'] - row['low']) - (row['high'] - row['close'])) / (row['high'] - row['low']) *
                abs((row['close'] - row['close_prev'])) / row['close_prev'] *
                (row['volume']*row['close']) / (row['ma5_prev'] * row['ma5_V'])) * 100 \
                + ((row['close'] - row['ma5_prev']) / row['ma5_prev'])/100 #Cộng thêm lượng này để tránh các trường hợp điểm dòng tiền bằng nhau gây trùng xếp hạng
    except ZeroDivisionError:
        return ((row['close'] - row['ma5_prev']) / row['ma5_prev'])/100 #Cộng thêm lượng này để tránh các trường hợp điểm dòng tiền bằng nhau gây trùng xếp hạng


In [107]:
#Tính cho toàn bộ lịch sử
raw_stock_score_dict = {}
for stock in full_stock_dict.keys():
    #Lọc ra các cột cần sử dụng
    temp_df = full_stock_dict[stock][['stock', 'date', 'period', 'count', 'high', 'low', 'close', 'volume', 'liquid_ratio', 'ma5', 'ma5_V']]
    #Tính điểm dòng tiền t0 và t5
    temp_df['ma5_prev'] = temp_df['ma5'].shift(-1)
    temp_df['close_prev'] = temp_df['close'].shift(-1)
    temp_df['t0_score'] = temp_df.apply(score_calculation, axis=1)
    temp_df['t5_score'] = temp_df['t0_score'][::-1].rolling(window=5, min_periods=1).mean()[::-1]

    temp_df = temp_df.drop(columns=['ma5_prev','close_prev'])
    #Gán lại temp_df cho dict
    raw_stock_score_dict[stock] = temp_df

In [108]:
#Tách điểm dòng tiền vào từng giai đoạn để tính xếp hạng
period_stock_score_dict = {}
for period in period_stock_df.columns[1:].tolist():
    period_stock_score_dict[period] = {k: v[(v['date'] >= period_map_dict[period][0]) & 
                                            (v['date'] <= period_map_dict[period][1])].reset_index(drop=True)
                                            for k, v in copy.deepcopy(raw_stock_score_dict).items()
                                            if k in period_stock_df[period].dropna().tolist()}

#Tính xếp hạng cổ phiếu cho từng giai đoạn
for period in period_stock_df.columns[1:].tolist():
    the_dict = period_stock_score_dict[period]

    #Tính xếp hạng cho cổ phiếu
    t0_ranking_df = period_date_series_dict[period].copy()
    t5_ranking_df = period_date_series_dict[period].copy()
    for stock in the_dict.keys():
        t0_ranking_df[stock] = the_dict[stock]['t0_score']
        t0_ranking_df.fillna(0, inplace=True)
        t5_ranking_df[stock] = the_dict[stock]['t5_score']
        t5_ranking_df.fillna(0, inplace=True)
    t0_ranking_df = t0_ranking_df.iloc[:,1:].rank(ascending=False, method='min', axis=1)
    t5_ranking_df = t5_ranking_df.iloc[:,1:].rank(ascending=False, method='min', axis=1)

    #Ghép xếp hạng vào bảng thông tin cổ phiếu
    for stock, df in the_dict.items():
        df['rank_t0'] = t0_ranking_df[stock]
        df['rank_t5'] = t5_ranking_df[stock]
        
        #Check xem xếp hạng T0 nằm trong top 10% hay không
        df['top_check'] = df.apply(lambda x: 1 if x['rank_t0'] <= x['count']*0.1 else 0, axis=1)

In [109]:
#Ghép lại bảng điểm dòng tiền cho tất cả giai đoạn
concat_stock_score_dict = {}
for stock in full_stock_classification_df['stock'].tolist():
    temp_df = pd.DataFrame()
    for period in period_stock_df.columns[1:].tolist():
        if stock in period_stock_df[period].tolist():
            temp_df = pd.concat([temp_df, period_stock_score_dict[period][stock]], axis=0)

    #Gán lại temp_df cho dict dòng tiền tổng
    concat_stock_score_dict[stock] = temp_df.reset_index(drop=True)

In [110]:
#Merge bảng và xử lý các giai đoạn không có cổ phiếu
full_stock_score_dict = {}
for stock, df in raw_stock_score_dict.items():
    df = df.merge(concat_stock_score_dict[stock][['date','rank_t0','rank_t5','top_check']], on='date', how='left')
    df['rank_t0'] = df.apply(lambda x: x['rank_t0'] if pd.notnull(x['rank_t0']) else x['count'], axis=1)
    df['rank_t5'] = df.apply(lambda x: x['rank_t5'] if pd.notnull(x['rank_t5']) else x['count'], axis=1)
    df['top_check'] = df['top_check'].fillna(0)

    #Đếm số phiên lọt top 10%
    df['top_count'] = df['top_check'][::-1].rolling(window=20, min_periods=1).sum()[::-1]

    full_stock_score_dict[stock] = df

#### Điểm dòng tiền nhóm cổ phiếu

- Dòng tiền vào nhóm cổ phiếu EOD

In [111]:
#Chỉnh sửa lại điểm dòng tiền t0 cho từng cổ phiếu với tác động của độ rộng từng nhóm
def adjust_score_by_breath(t0_score, ratio_column):
    adjusted_score = []
    for score, ratio in zip(t0_score, ratio_column):
        if score >= 0:
            adjusted_score.append(score*ratio)
        else:
            adjusted_score.append(score*(1-ratio))
    return adjusted_score

#Hàm điều chỉnh điểm dòng tiền của cổ phiếu tránh sự đột biến khi đóng góp vào nhóm chung
def adjust_score_for_smooth(row, column_name, max_percent, mark):
    origin_score = row[column_name]
    
    if abs(origin_score) > row['total'] * max_percent:

        sum_abs = row['total'] - abs(row[column_name])
        fixed_score = sum_abs / (1 - max_percent) - sum_abs

        if origin_score >= 0:
            return fixed_score
        else:
            return -fixed_score
    else:
        mark[0] = 0
        return origin_score

#Áp dụng hàm điều chỉnh điểm phía trên vào các nhóm cổ phiếu, việc này lặp lại nhiều lần cho tới khi triệt tiêu sự đột biến
def apply_smooth_score(score_dict, group_type, type_name):

    if type_name == 'itd':
        initial_score_df = time_series.copy()
    elif type_name == 'eod':
        initial_score_df = date_series.copy()

    if group_type == 'all_stock':
        key_list = all_stock_key_list
    elif group_type == 'industry_perform':
        key_list = [key for key, value in group_map_dict.items() if value == 'hs']
    elif group_type == 'marketcap_group':
        key_list = [key for key, value in group_map_dict.items() if value == 'cap'] 
    elif group_type == 'industry_name':
        key_list = [key for key, value in group_map_dict.items() if value in ['A', 'B', 'C', 'D']]

    for key in key_list:
        score_df = initial_score_df.copy()
        if group_type == 'all_stock':
            stock_list = stock_classification_df['stock'].tolist()
        else:
            stock_list = [stock for stock in stock_classification_df[stock_classification_df[group_type]==key]['stock'].dropna().tolist()]
        for stock in stock_list:
            try: score_df[stock] = score_dict[stock][f't0_score']
            except: pass

        max_percent = max(0.1, min(5*(1/len(stock_list)), 0.5))
        score_df['total'] = score_df.iloc[:, 1:].abs().sum(axis=1)

        mark = [1]
        while True:
            if mark[0] == 1:
                for stock in stock_list:
                    score_df[stock] = score_df.iloc[:, 1:].apply(adjust_score_for_smooth, axis=1, args=(stock, max_percent, mark))
            if mark[0] == 0: break

        for stock in stock_list:
            try: score_dict[stock][f't0_{group_type}'] = score_df[stock]
            except: pass

In [112]:
#Thêm các cột dòng tiền đóng góp vào các nhóm cổ phiếu vào các dict period (đã loại bỏ đột biến)
for group_type in ['all_stock','industry_name','industry_perform','marketcap_group']:
    apply_smooth_score(eod_score_dict, group_type, 'eod')

In [113]:
#Tính độ rộng cho từng phiên phục vụ cho việc điều chỉnh điểm dòng tiền
temp_df = date_series.copy()
for stock, df in eod_score_dict.items():
    temp_df[stock] = eod_score_dict[stock]['t0_score']
temp_df.iloc[:,1:] = temp_df.iloc[:,1:].applymap(lambda x: 1 if x > 0 else 0)

eod_market_breath = date_series.copy()

industry_name_breadth_dict = {}
for key in eod_industry_name.keys():
    stock_list = stock_classification_df[stock_classification_df['industry_name']==key]['stock'].tolist()
    industry_name_breadth_dict[key] = temp_df[['date'] + [columns for columns in stock_list]]
    eod_market_breath[key] = industry_name_breadth_dict[key].iloc[:,1:].sum(axis=1)/len(stock_list)

industry_perform_breadth_dict = {}
for key in eod_industry_perform.keys():
    stock_list = stock_classification_df[stock_classification_df['industry_perform']==key]['stock'].tolist()
    industry_perform_breadth_dict[key] = temp_df[['date'] + [columns for columns in stock_list]]
    eod_market_breath[key] = industry_perform_breadth_dict[key].iloc[:,1:].sum(axis=1)/len(stock_list)

marketcap_group_breadth_dict = {}
for key in eod_marketcap_group.keys():
    stock_list = stock_classification_df[stock_classification_df['marketcap_group']==key]['stock'].tolist()
    marketcap_group_breadth_dict[key] = temp_df[['date'] + [columns for columns in stock_list]]
    eod_market_breath[key] = marketcap_group_breadth_dict[key].iloc[:,1:].sum(axis=1)/len(stock_list)

all_stock_breadth_dict = {}
for key in eod_all_stock.keys():
    stock_list = stock_classification_df['stock'].tolist()
    all_stock_breadth_dict[key] = temp_df[['date'] + [columns for columns in stock_list]]
    eod_market_breath[key] = all_stock_breadth_dict[key].iloc[:,1:].sum(axis=1)/len(stock_list)

#Chỉnh sửa lại điểm dòng tiền t0 cho từng cổ phiếu với tác động của độ rộng từng nhóm
for stock, df in eod_score_dict.items():
    name_of_industry_name = stock_classification_df[stock_classification_df['stock']==stock]['industry_name'].item()
    name_of_industry_perform = stock_classification_df[stock_classification_df['stock']==stock]['industry_perform'].item()
    name_of_marketcap_group = stock_classification_df[stock_classification_df['stock']==stock]['marketcap_group'].item()

    df[f't0_industry_name'] = adjust_score_by_breath(df['t0_industry_name'], eod_market_breath[name_of_industry_name])
    df[f't0_industry_perform'] = adjust_score_by_breath(df['t0_industry_perform'], eod_market_breath[name_of_industry_perform])
    df[f't0_marketcap_group'] = adjust_score_by_breath(df['t0_marketcap_group'], eod_market_breath[name_of_marketcap_group])
    df[f't0_all_stock'] = adjust_score_by_breath(df['t0_all_stock'], eod_market_breath['all_stock'])

In [114]:
def mean_of_net_values(df):
    net_values = df  # Lọc ra các giá trị âm
    return net_values.mean(axis=1)

# Tạo bảng dữ liệu điểm dòng tiền cho các nhóm cổ phiếu
eod_group_score_df_net = date_series.copy()

# Thêm cột điểm dòng tiền toàn bộ cổ phiếu
for nganh in eod_all_stock.keys():
    score_df = date_series.copy()
    for stock in stock_classification_df['stock']:
        score_df[stock] = eod_score_dict[stock]['t0_all_stock']
    score_df['total'] = mean_of_net_values(score_df.iloc[:, 1:])
    eod_group_score_df_net[nganh] = score_df['total']

# Thêm các cột điểm dòng tiền ngành
eod_industry_name_score_df = date_series.copy()
for nganh in eod_industry_name.keys():
    score_df = date_series.copy()
    for stock in stock_classification_df[stock_classification_df['industry_name']==nganh]['stock']:
        score_df[stock] = eod_score_dict[stock]['t0_industry_name']
    score_df['total'] = mean_of_net_values(score_df.iloc[:, 1:])
    eod_group_score_df_net[nganh] = score_df['total']

# Thêm các cột điểm dòng tiền nhóm hiệu suất
eod_industry_perform_score_df = date_series.copy()
for group in eod_industry_perform.keys():
    score_df = date_series.copy()
    for stock in stock_classification_df[stock_classification_df['industry_perform']==group]['stock']:
        score_df[stock] = eod_score_dict[stock]['t0_industry_perform']
    score_df['total'] = mean_of_net_values(score_df.iloc[:, 1:])
    eod_group_score_df_net[group] = score_df['total']

# Thêm các cột điểm dòng tiền nhóm vốn hoá
eod_marketcap_group_score_df = date_series.copy()
for marketcap in eod_marketcap_group.keys():
    score_df = date_series.copy()
    for stock in stock_classification_df[stock_classification_df['marketcap_group']==marketcap]['stock']:
        score_df[stock] = eod_score_dict[stock]['t0_marketcap_group']
    score_df['total'] = mean_of_net_values(score_df.iloc[:, 1:])
    eod_group_score_df_net[marketcap] = score_df['total']

eod_group_score_df_net = eod_group_score_df_net.fillna(0)

#Lấy ra dữ liệu cho quý hiện tại
eod_group_score_df_net = eod_group_score_df_net[eod_group_score_df_net['date'] >= current_quarter_span[0]]

#Đọc dữ liệu các quý trong quá khữ
period_group_score_df_net = pd.read_excel("../period_data/period_processed_data.xlsx", sheet_name='full_group_score_df')

#Ghép lại thành abrng hoàn chỉnh
full_group_score_df_net = pd.concat([eod_group_score_df_net, period_group_score_df_net], axis=0).reset_index(drop=True)

In [115]:
def mean_of_positive_values(df, length):
    positive_values = df[df >= 0]  # Lọc ra các giá trị âm
    return positive_values.sum(axis=1)/length

# Tạo bảng dữ liệu điểm dòng tiền cho các nhóm cổ phiếu
eod_group_score_df_positive = date_series.copy()

# Thêm cột điểm dòng tiền toàn bộ cổ phiếu
for nganh in eod_all_stock.keys():
    score_df = date_series.copy()
    temp_stock_list = stock_classification_df['stock']
    for stock in temp_stock_list:
        score_df[stock] = eod_score_dict[stock]['t0_all_stock']
    score_df['total'] = mean_of_positive_values(score_df.iloc[:, 1:], len(temp_stock_list))
    eod_group_score_df_positive[nganh] = score_df['total']

# Thêm các cột điểm dòng tiền ngành
eod_industry_name_score_df = date_series.copy()
for nganh in eod_industry_name.keys():
    score_df = date_series.copy()
    temp_stock_list = stock_classification_df[stock_classification_df['industry_name']==nganh]['stock']
    for stock in temp_stock_list:
        score_df[stock] = eod_score_dict[stock]['t0_industry_name']
    score_df['total'] = mean_of_positive_values(score_df.iloc[:, 1:], len(temp_stock_list))
    eod_group_score_df_positive[nganh] = score_df['total']

# Thêm các cột điểm dòng tiền nhóm hiệu suất
eod_industry_perform_score_df = date_series.copy()
for group in eod_industry_perform.keys():
    score_df = date_series.copy()
    temp_stock_list = stock_classification_df[stock_classification_df['industry_perform']==group]['stock']
    for stock in temp_stock_list:
        score_df[stock] = eod_score_dict[stock]['t0_industry_perform']
    score_df['total'] = mean_of_positive_values(score_df.iloc[:, 1:], len(temp_stock_list))
    eod_group_score_df_positive[group] = score_df['total']

# Thêm các cột điểm dòng tiền nhóm vốn hoá
eod_marketcap_group_score_df = date_series.copy()
for marketcap in eod_marketcap_group.keys():
    score_df = date_series.copy()
    temp_stock_list = stock_classification_df[stock_classification_df['marketcap_group']==marketcap]['stock']
    for stock in temp_stock_list:
        score_df[stock] = eod_score_dict[stock]['t0_marketcap_group']
    score_df['total'] = mean_of_positive_values(score_df.iloc[:, 1:], len(temp_stock_list))
    eod_group_score_df_positive[marketcap] = score_df['total']

eod_group_score_df_positive = eod_group_score_df_positive.fillna(0)

#Lấy ra dữ liệu cho quý hiện tại
eod_group_score_df_positive = eod_group_score_df_positive[eod_group_score_df_positive['date'] >= current_quarter_span[0]]

#Đọc dữ liệu các quý trong quá khữ
period_group_score_df_positive = pd.read_excel("../period_data/period_processed_data.xlsx", sheet_name='full_group_score_positive_df')

#Ghép lại thành abrng hoàn chỉnh
full_group_score_df_positive = pd.concat([eod_group_score_df_positive, period_group_score_df_positive], axis=0).reset_index(drop=True)

In [116]:
def mean_of_negative_values(df, length):
    negative_values = df[df < 0]  # Lọc ra các giá trị âm
    return negative_values.sum(axis=1)/length

# Tạo bảng dữ liệu điểm dòng tiền cho các nhóm cổ phiếu
eod_group_score_df_negative = date_series.copy()

# Thêm cột điểm dòng tiền toàn bộ cổ phiếu
for nganh in eod_all_stock.keys():
    score_df = date_series.copy()
    temp_stock_list = stock_classification_df['stock']
    for stock in temp_stock_list:
        score_df[stock] = eod_score_dict[stock]['t0_all_stock']
    score_df['total'] = mean_of_negative_values(score_df.iloc[:, 1:], len(temp_stock_list))
    eod_group_score_df_negative[nganh] = score_df['total']

# Thêm các cột điểm dòng tiền ngành
eod_industry_name_score_df = date_series.copy()
for nganh in eod_industry_name.keys():
    score_df = date_series.copy()
    temp_stock_list = stock_classification_df[stock_classification_df['industry_name']==nganh]['stock']
    for stock in temp_stock_list:
        score_df[stock] = eod_score_dict[stock]['t0_industry_name']
    score_df['total'] = mean_of_negative_values(score_df.iloc[:, 1:], len(temp_stock_list))
    eod_group_score_df_negative[nganh] = score_df['total']

# Thêm các cột điểm dòng tiền nhóm hiệu suất
eod_industry_perform_score_df = date_series.copy()
for group in eod_industry_perform.keys():
    score_df = date_series.copy()
    temp_stock_list = stock_classification_df[stock_classification_df['industry_perform']==group]['stock']
    for stock in temp_stock_list:
        score_df[stock] = eod_score_dict[stock]['t0_industry_perform']
    score_df['total'] = mean_of_negative_values(score_df.iloc[:, 1:], len(temp_stock_list))
    eod_group_score_df_negative[group] = score_df['total']

# Thêm các cột điểm dòng tiền nhóm vốn hoá
eod_marketcap_group_score_df = date_series.copy()
for marketcap in eod_marketcap_group.keys():
    score_df = date_series.copy()
    temp_stock_list = stock_classification_df[stock_classification_df['marketcap_group']==marketcap]['stock']
    for stock in temp_stock_list:
        score_df[stock] = eod_score_dict[stock]['t0_marketcap_group']
    score_df['total'] = mean_of_negative_values(score_df.iloc[:, 1:], len(temp_stock_list))
    eod_group_score_df_negative[marketcap] = score_df['total']

eod_group_score_df_negative = eod_group_score_df_negative.fillna(0)

#Lấy ra dữ liệu cho quý hiện tại
eod_group_score_df_negative = eod_group_score_df_negative[eod_group_score_df_negative['date'] >= current_quarter_span[0]]

#Đọc dữ liệu các quý trong quá khữ
period_group_score_df_negative = pd.read_excel("../period_data/period_processed_data.xlsx", sheet_name='full_group_score_negative_df')

#Ghép lại thành abrng hoàn chỉnh
full_group_score_df_negative = pd.concat([eod_group_score_df_negative, period_group_score_df_negative], axis=0).reset_index(drop=True)

#### Tính dữ liệu cho MS

In [117]:
def transform_ms(stock_group):
    stock_dict = copy.deepcopy(stock_group)

    # Prepare a base date DataFrame from date_series
    dates_df = pd.DataFrame(date_series['date'].tolist(), columns=['date'])
    
    for group_name, stocks in stock_dict.items():
        # Initialize a DataFrame for group trends
        group_trends = dates_df.copy()

        # Compute trends across stocks
        for trend in ['trend_5p', 'trend_20p', 'trend_60p', 'trend_120p', 'trend_240p', 'trend_480p']:
            # Concatenate all trend data for current trend across all stocks
            trend_data = pd.concat([stocks[stock][trend] for stock in stocks], axis=1)
            trend_data.fillna(0, inplace=True)
            
            # Calculate the sum and percent for the trend
            sum_trend = trend_data.sum(axis=1)
            percent_trend = sum_trend / len(stocks)
            
            # Add to group trends DataFrame
            group_trends[f'{trend}'] = percent_trend

        stock_dict[group_name] = group_trends[group_trends['date'] >= current_quarter_span[0]].sort_values('date', ascending=False)

    return stock_dict

In [118]:
#Tính toán các biểu đồ MS cho các nhóm cổ phiếu
all_stock_ms = transform_ms(eod_all_stock)
industry_name_ms = transform_ms(eod_industry_name)
industry_perform_ms = transform_ms(eod_industry_perform)
marketcap_group_ms = transform_ms(eod_marketcap_group)

#Gộp tất cả biểu đồ MS vào 1 bảng
temp_market_ms = pd.DataFrame()
for item in [all_stock_ms, industry_name_ms, industry_perform_ms, marketcap_group_ms]:
    for group, df in item.items():
        df['name'] = group
        temp_market_ms = pd.concat([temp_market_ms, df], axis=0)

#Lấy dữ liệu lịch sử MS đã tính toán
period_market_ms = pd.read_excel("../period_data/period_processed_data.xlsx", sheet_name='full_group_ms_chart_df')

#Ghép bảng dữ liệu lịch sử với dữ liệu của quý này
full_market_ms = pd.concat([temp_market_ms, period_market_ms], axis=0).sort_values('date', ascending=False).reset_index(drop=True)

#### Tính dữ liệu cho group price index

In [119]:
def calculate_total_change(stock_group, name, price_index_date_series):
    period_index_df = price_index_date_series.copy()

    for stock, df in stock_group[name].items():
        period_index_df[stock] = df['close']
        period_index_df[stock] = period_index_df[stock][::-1].pct_change()[::-1]

    period_index_df['total_change'] = period_index_df.iloc[:,1:].sum(axis=1)
    period_index_df['total_change'] = ((period_index_df['total_change']/len(stock_group[name]))*100)
    period_index_df['total_change'] = period_index_df['total_change']*10

    return period_index_df['total_change']

In [120]:
#Lấy dữ liệu lịch sử group price change đã tính toán
period_group_price_change = pd.read_excel("../period_data/period_processed_data.xlsx", sheet_name='full_group_price_change_df')

#Tính dữ liệu group price change của quý hiện tại
temp_group_price_change = date_series.copy()
for key in all_stock_key_list:
    temp_group_price_change[key] = calculate_total_change(eod_all_stock, key, date_series)

for key in industry_name_list:
    temp_group_price_change[key] = calculate_total_change(eod_industry_name, key, date_series)

for key in industry_perform_list:
    temp_group_price_change[key] = calculate_total_change(eod_industry_perform, key, date_series)

for key in marketcap_group_list:
    temp_group_price_change[key] = calculate_total_change(eod_marketcap_group, key, date_series)

temp_group_price_change = temp_group_price_change[temp_group_price_change['date'] >= current_quarter_span[0]]

#Ghép dữ liệu thay đổi index các nhóm cổ phiếu
group_price_index_df = pd.concat([temp_group_price_change, period_group_price_change]).sort_values('date', ascending=False).reset_index(drop=True)

for key in group_stock_key_list:
    group_price_index_df[key] = group_price_index_df[key][::-1].cumsum()[::-1] + 1000

#Thêm cột VNINDEX để tham chiếu
group_price_index_df = group_price_index_df.merge(
    eod_index_dict['VNINDEX'][['date', 'close']].rename(columns={'close': 'VNINDEX'}),
    on='date',
    how='left'
)

#### Các hàm tính toán sử dụng trong phân bổ vốn

In [121]:
def calculate_final_portion(df):
    df['final_portion'] = df['portion_raw']
    for i in range(len(df) - 2, -1, -1):
        if df['portion_phase'].iloc[i] == 1:
            if df['final_portion'].iloc[i+1] > df['final_portion'].iloc[i]:
                df['final_portion'].iloc[i] = df['final_portion'].iloc[i+1]
        if df['portion_phase'].iloc[i] == -1:
            if df['final_portion'].iloc[i+1] < df['final_portion'].iloc[i]:
                df['final_portion'].iloc[i] = df['final_portion'].iloc[i+1]
        if df['portion_phase'].iloc[i] == 2:
            if df['final_portion'].iloc[i+1] != df['final_portion'].iloc[i]:
                df['final_portion'].iloc[i] = df['final_portion'].iloc[i+1]
        if df['portion_phase'].iloc[i] == -2:
            if df['final_portion'].iloc[i+1] != df['final_portion'].iloc[i]:
                df['final_portion'].iloc[i] = df['final_portion'].iloc[i+1]
    return df['final_portion']

def adjust_portion_T3(df):
    df['final_portion'] = df['final_portion']
    for i in range(len(df) - 4, -1, -1):
        if (df['final_portion'].iloc[i+1] > df['final_portion'].iloc[i+2]) & (df['final_portion'].iloc[i+1] > df['final_portion'].iloc[i]):
            df['final_portion'].iloc[i] = df['final_portion'].iloc[i+1]
        elif (df['final_portion'].iloc[i+2] > df['final_portion'].iloc[i+3]) & (df['final_portion'].iloc[i+1] > df['final_portion'].iloc[i]):
            df['final_portion'].iloc[i] = df['final_portion'].iloc[i+1]
    return df['final_portion']

In [122]:
#Tính toán tín hiệu của MS
def calculate_ms(ms_df):
    index_ms = ms_df

    index_ms['5p_shift1'] = index_ms['trend_5p'].shift(-1)
    index_ms['5p_shift2'] = index_ms['trend_5p'].shift(-2)
    # index_ms['5p_shift4'] = index_ms['trend_5p'].shift(-4)
    index_ms['5p_shift4'] = index_ms['trend_5p'][::-1].rolling(window=5, min_periods=1).mean()[::-1].shift(-1)
    
    index_ms['20p_shift1'] = index_ms['trend_20p'].shift(-1)
    index_ms['20p_shift2'] = index_ms['trend_20p'].shift(-2)
    # index_ms['20p_shift4'] = index_ms['trend_20p'].shift(-4)
    index_ms['20p_shift4'] = index_ms['trend_20p'][::-1].rolling(window=5, min_periods=1).mean()[::-1].shift(-1)

    index_ms['60p_shift1'] = index_ms['trend_60p'].shift(-1)
    index_ms['60p_shift2'] = index_ms['trend_60p'].shift(-2)

    #Check điểm mua
    index_ms['5p_upcheck1'] = index_ms.apply(lambda x: 1 if (x['trend_5p'] - x['5p_shift1']) >= 0 else 0, axis = 1)
    index_ms['5p_upcheck4'] = index_ms.apply(lambda x: 1 if (x['trend_5p'] - x['5p_shift4']) >= 0 else 0, axis = 1)
    index_ms['5p_upcheck'] = index_ms.apply(lambda x: 1 if (x['5p_upcheck1'] == 1) & (x['5p_upcheck4'] == 1) else 0, axis = 1)

    index_ms['20p_upcheck1'] = index_ms.apply(lambda x: 1 if (x['trend_20p'] - x['20p_shift1']) >= 0 else 0, axis = 1)
    index_ms['20p_upcheck4'] = index_ms.apply(lambda x: 1 if (x['trend_20p'] - x['20p_shift4']) >= 0 else 0, axis = 1)
    index_ms['20p_upcheck'] = index_ms.apply(lambda x: 1 if (x['20p_upcheck1'] == 1) & (x['20p_upcheck4'] == 1) else 0, axis = 1)

    index_ms['60p_upcheck1'] = index_ms.apply(lambda x: 1 if (x['trend_60p'] - x['60p_shift1']) >= 0 else 0, axis = 1)
    index_ms['60p_upcheck2'] = index_ms.apply(lambda x: 1 if (x['trend_60p'] - x['60p_shift2']) >= 0 else 0, axis = 1)
    index_ms['60p_upcheck'] = index_ms.apply(lambda x: 1 if (x['60p_upcheck1'] == 1) & (x['60p_upcheck2'] == 1) else 0, axis = 1)

    index_ms['up_check'] = index_ms.apply(lambda x: 1 if (x['5p_upcheck'] == 1) & (x['20p_upcheck'] == 1) & (x['60p_upcheck'] == 1) & 
                                                        (x['trend_5p'] < 0.9) & (x['trend_20p'] < 0.9) &
                                                        (((x['trend_5p'] - x['5p_shift1']) > 0.5) | ((x['trend_5p'] > 0.3) & (x['trend_20p'] > 0.2)))

                                                   else (2 if (x['trend_5p'] >= 0.9) | (x['trend_20p'] >= 0.9) else 0)
                                                        , axis = 1)

    #Check điểm bán
    index_ms['5p_downcheck1'] = index_ms.apply(lambda x: 1 if (x['trend_5p'] - x['5p_shift1']) < 0 else 0, axis = 1)
    index_ms['5p_downcheck2'] = index_ms.apply(lambda x: 1 if (x['trend_5p'] - x['5p_shift2']) < 0 else 0, axis = 1)
    index_ms['5p_downcheck'] = index_ms.apply(lambda x: 1 if (x['5p_downcheck1'] == 1) & (x['5p_downcheck2'] == 1) else 0, axis = 1)

    index_ms['20p_downcheck1'] = index_ms.apply(lambda x: 1 if (x['trend_20p'] - x['20p_shift1']) < 0 else 0, axis = 1)
    index_ms['20p_downcheck2'] = index_ms.apply(lambda x: 1 if (x['trend_20p'] - x['20p_shift2']) < 0 else 0, axis = 1)
    index_ms['20p_downcheck'] = index_ms.apply(lambda x: 1 if (x['20p_downcheck1'] == 1) & (x['20p_downcheck2'] == 1) else 0, axis = 1)

    index_ms['down_check'] = index_ms.apply(lambda x: 1 if (x['5p_downcheck'] == 1) & (x['20p_downcheck'] == 1) & 
                                                            ((x['trend_5p'] > 0.1) | (x['trend_20p'] > 0.1)) & 
                                                            ((((x['trend_5p'] - x['5p_shift4']) < -0.5) & (x['trend_5p'] < 0.3)) | ((x['trend_5p'] < 0.4) & (x['trend_20p'] < 0.5)))

                                                        else (2 if (x['trend_5p'] <= 0.1) & (x['trend_20p'] <= 0.1) else 0)
                                                        , axis = 1)

    return index_ms

In [123]:
def calculate_portion_phase(df):
    df['portion_phase'] = df.apply(
    lambda x: 2 if x['up_check'] == 2 else 
            (1 if (x['up_check'] == 1) & (x['portion_raw'] > 0) else
            (-1 if x['down_check'] == 1 else
            (-2 if x['down_check'] == 2 else 
            None))), 
    axis=1)

    #Xoá đi các tín hiệu mua nhưng lại ngược chiều với chiều biến động của portion_raw
    for i in range(len(df) - 2, -1, -1):
        if (df['portion_phase'].iloc[i] == 1) & (df['portion_raw'].iloc[i] < df['portion_raw'].iloc[i+1]):
            df.loc[i, 'portion_phase'] = None
        elif (df['portion_phase'].iloc[i] == 1) & (df['A_portion'].iloc[i] == 0):
            df.loc[i, 'portion_phase'] = None

    #Thêm trạng thái đầu tiên là mua vào để khi fill ko bị NaN
    df['portion_phase'].iloc[-1] = 1
    df['portion_phase'] = df['portion_phase'].bfill()

    return df['portion_phase']

#### Tín hiệu mua bán tổng

In [124]:
#Tính phân bổ vốn hoá tổng chưa điều chỉnh
phan_bo_von_raw = pd.DataFrame()
phan_bo_von_raw['date'] = date_series['date']
phan_bo_von_raw[['A+','B+','C+','D+']] = full_group_score_df_positive[['A','B','C','D']][::-1].rolling(window=5, min_periods=1).mean()[::-1]
phan_bo_von_raw[['A-','B-','C-','D-']] = full_group_score_df_negative[['A','B','C','D']][::-1].rolling(window=5, min_periods=1).mean().abs()[::-1]

phan_bo_von_raw['A_raw'] = (phan_bo_von_raw['A+'] - phan_bo_von_raw['A-']) / (phan_bo_von_raw['A+'] + phan_bo_von_raw['A-'])
phan_bo_von_raw['B_raw'] = (phan_bo_von_raw['B+'] - phan_bo_von_raw['B-']) / (phan_bo_von_raw['B+'] + phan_bo_von_raw['B-'])
phan_bo_von_raw['C_raw'] = (phan_bo_von_raw['C+'] - phan_bo_von_raw['C-']) / (phan_bo_von_raw['C+'] + phan_bo_von_raw['C-'])
phan_bo_von_raw['D_raw'] = (phan_bo_von_raw['D+'] - phan_bo_von_raw['D-']) / (phan_bo_von_raw['D+'] + phan_bo_von_raw['D-'])
phan_bo_von_raw = phan_bo_von_raw.fillna(0)

phan_bo_von_raw['A_portion'] = phan_bo_von_raw['A_raw'].apply(lambda x: x*0.3 if x*0.3 > 0.1 else 0)
phan_bo_von_raw['B_portion'] = phan_bo_von_raw['B_raw'].apply(lambda x: x*0.3 if x > 0 else 0)
phan_bo_von_raw['C_portion'] = phan_bo_von_raw['C_raw'].apply(lambda x: x*0.3 if x > 0 else 0)
phan_bo_von_raw['D_portion'] = phan_bo_von_raw['D_raw'].apply(lambda x: x*0.1 if x > 0 else 0)

phan_bo_von_raw['sum'] = phan_bo_von_raw[['A_portion','B_portion','C_portion','D_portion']].sum(axis=1).apply(lambda x: x if x >= 0.2 else 0)

In [125]:
#Điều chỉnh lại phân bổ vốn hoá tổng theo MS
index_ms = calculate_ms(full_market_ms[full_market_ms['name']=='all_stock'][['date','trend_5p','trend_20p','trend_60p']]).sort_values('date', ascending=False).reset_index(drop=True)

phan_bo_von_final = pd.DataFrame()
phan_bo_von_final[['date','A_portion','B_portion','C_portion','D_portion','portion_raw']] = phan_bo_von_raw[['date','A_portion','B_portion','C_portion','D_portion','sum']]

#Tạo các giai đoạn mua và bán
phan_bo_von_final = phan_bo_von_final.merge(index_ms[['date','up_check','down_check']], on='date', how='left')
phan_bo_von_final['portion_phase'] = calculate_portion_phase(phan_bo_von_final)

# Điều chỉnh tỉ trọng cuối cùng theo các giai đoạn mua bán
phan_bo_von_final['final_portion'] = calculate_final_portion(phan_bo_von_final)

#Điều chỉnh để nếu có tăng giảm trong phân bổ tỉ tăng lên 1 và giảm về 0 luôn
phan_bo_von_final['final_portion'] = phan_bo_von_final.apply(lambda x: 1 if (x['portion_phase'] > 0) & (x['final_portion'] > 0)  else x['final_portion'], axis=1)
phan_bo_von_final['final_portion'] = phan_bo_von_final.apply(lambda x: 0 if (x['portion_phase'] < 0) & (x['final_portion'] < 1)  else x['final_portion'], axis=1)

#Điều chỉnh lại để tránh mua bán ko kịp T2
phan_bo_von_final['final_portion'] = adjust_portion_T3(phan_bo_von_final)

#Điều chỉnh lại tỉ trọng 4 nhóm hiệu suất về cố định 532
phan_bo_von_final['A_portion'] = phan_bo_von_final.apply(lambda x: 0.4 if x['final_portion'] == 1 else 0, axis=1)
phan_bo_von_final['B_portion'] = phan_bo_von_final.apply(lambda x: 0.3 if x['final_portion'] == 1 else 0, axis=1)
phan_bo_von_final['C_portion'] = phan_bo_von_final.apply(lambda x: 0.3 if x['final_portion'] == 1 else 0, axis=1)
phan_bo_von_final['D_portion'] = 0

#### Phân bổ vốn theo ngành và HS

In [126]:
#Tính toán thay đổi index dựa theo tỉ trọng
def calculate_group_total_change(row, nganh_list):
    total_change = 0
    for nganh in nganh_list:
        total_change += row[nganh + '_portion'] * row[nganh + '_index_change']

    return total_change

In [127]:
#Tính toán tín hiệu của MS
def calculate_group_ms(ms_df):
    index_ms = ms_df

    index_ms['5p_shift1'] = index_ms['trend_5p'].shift(-1)
    index_ms['5p_shift2'] = index_ms['trend_5p'].shift(-2)
    index_ms['5p_shift4'] = index_ms['trend_5p'].shift(-4)
    index_ms['20p_shift1'] = index_ms['trend_20p'].shift(-1)
    index_ms['20p_shift2'] = index_ms['trend_20p'].shift(-2)
    index_ms['20p_shift4'] = index_ms['trend_20p'].shift(-4)

    #Check điểm bán
    index_ms['5p_downcheck1'] = index_ms.apply(lambda x: 1 if (x['trend_5p'] - x['5p_shift1']) < 0 else 0, axis = 1)
    index_ms['5p_downcheck2'] = index_ms.apply(lambda x: 1 if (x['trend_5p'] - x['5p_shift2']) < 0 else 0, axis = 1)
    index_ms['5p_downcheck'] = index_ms.apply(lambda x: 1 if (x['5p_downcheck1'] == 1) & (x['5p_downcheck2'] == 1) else 0, axis = 1)

    index_ms['20p_downcheck1'] = index_ms.apply(lambda x: 1 if (x['trend_20p'] - x['20p_shift1']) < 0 else 0, axis = 1)
    index_ms['20p_downcheck2'] = index_ms.apply(lambda x: 1 if (x['trend_20p'] - x['20p_shift2']) < 0 else 0, axis = 1)
    index_ms['20p_downcheck'] = index_ms.apply(lambda x: 1 if (x['20p_downcheck1'] == 1) & (x['20p_downcheck2'] == 1) else 0, axis = 1)

    index_ms['down_check'] = index_ms.apply(lambda x: 1 if (x['5p_downcheck'] == 1) & (x['20p_downcheck'] == 1) & 
                                                            (((x['trend_5p'] - x['5p_shift1']) < -0.5) | 
                                                            (x['trend_5p'] < 0.3) | 
                                                            ((x['trend_5p'] < 0.4) & (x['trend_20p'] < 0.5)))
                                                            else 0, axis = 1)
    
    #Check điểm mua
    index_ms['5p_upcheck1'] = index_ms.apply(lambda x: 1 if (x['trend_5p'] - x['5p_shift1']) >= 0 else 0, axis = 1)
    index_ms['5p_upcheck4'] = index_ms.apply(lambda x: 1 if (x['trend_5p'] - x['5p_shift4']) >= 0 else 0, axis = 1)
    index_ms['5p_upcheck'] = index_ms.apply(lambda x: 1 if (x['5p_upcheck1'] == 1) & (x['5p_upcheck4'] == 1) else 0, axis = 1)

    index_ms['20p_upcheck'] = index_ms.apply(lambda x: 1 if (x['trend_20p'] - x['20p_shift4']) >= 0 else 0, axis = 1)

    index_ms['up_check'] = index_ms.apply(lambda x: 1 if (x['5p_upcheck'] == 1) & (x['20p_upcheck'] == 1) &
                                                        (((x['trend_5p'] - x['5p_shift1']) > 0.5) | 
                                                        ((x['trend_5p'] > 0.2) & (x['trend_20p'] > 0)))
                                                        else 0, axis = 1)
    return index_ms

def calculate_portion_phase(df):
    df['portion_phase'] = df.apply(
    lambda x: 1 if (x['up_check'] == 1) & (x['portion_raw'] > 0) else
            (0 if x['down_check'] == 1 else
            None), 
    axis=1)

    #Xoá đi các tín hiệu mua nhưng lại ngược chiều với chiều biến động của portion_raw
    for i in range(len(df) - 2, -1, -1):
        if (df['portion_phase'].iloc[i] == 1) & (df['portion_raw'].iloc[i] < df['portion_raw'].iloc[i+1]):
            df.loc[i, 'portion_phase'] = None

    #Xoá đi các tín hiệu mua nhưng đang trong giai đoạn bán của thị trường chung
    for i in range(len(df) - 1, -1, -1):
        if (df['portion_phase'].iloc[i] == 1) & (df['total_portion'].iloc[i] == 0):
            df.loc[i, 'portion_phase'] = None

    #Thêm trạng thái đầu tiên là mua vào để khi fill ko bị NaN
    df['portion_phase'].iloc[-1] = 1
    df['portion_phase'] = df['portion_phase'].bfill()

    return df['portion_phase']

def calculate_final_portion_nhom(df):
    df['final_portion'] = df['portion_raw']
    for i in range(len(df) - 2, -1, -1):
        if df['portion_phase'].iloc[i] == 1:
            if df['final_portion'].iloc[i] == 0:
                df['final_portion'].iloc[i] = df['final_portion'].iloc[i+1]
        if df['portion_phase'].iloc[i] == 0:
                df['final_portion'].iloc[i] = 0
    return df['final_portion']

In [128]:
def adjust_group_portion_T3(df, column_list):
    for i in range(len(df) - 4, -1, -1):

        prev3_nganh_series = df[column_list].iloc[i+3]
        prev3_nganh_list = prev3_nganh_series[prev3_nganh_series > 0].index.tolist()

        prev2_nganh_series = df[column_list].iloc[i+2]
        prev2_nganh_list = prev2_nganh_series[prev2_nganh_series > 0].index.tolist()

        prev1_nganh_series = df[column_list].iloc[i+1]
        prev1_nganh_list = prev1_nganh_series[prev1_nganh_series > 0].index.tolist()

        try: prev1_nganh_1 = prev1_nganh_list[0]
        except: prev1_nganh_1 = None
        try: prev1_nganh_2 = prev1_nganh_list[1]
        except: prev1_nganh_2 = None

        #Kiểm tra xem ngành 1 đã bán được chưa
        if (prev1_nganh_1 in prev2_nganh_list) & (prev1_nganh_1 in prev3_nganh_list):
            check_nganh_1 = True
        else: check_nganh_1 = False

        #Kiểm tra xem ngành 2 đã bán được chưa
        if (prev1_nganh_2 in prev2_nganh_list) & (prev1_nganh_2 in prev3_nganh_list):
            check_nganh_2 = True
        else: check_nganh_2 = False

        #Tạo danh sách ngành hiện tại khi chưa hiệu chỉnh
        current_nganh_series = df[column_list].iloc[i]
        current_nganh_list = current_nganh_series[current_nganh_series > 0].index.tolist()

        #Hiệu chỉnh danh sách ngành cho đúng T3
        if len(current_nganh_list) == 2:
            if ((check_nganh_1 == False) & (prev1_nganh_1 != None)) & ((check_nganh_2 == False) & (prev1_nganh_2 != None)):
                final_nganh_list = prev1_nganh_list
            elif ((check_nganh_1 == False) & (prev1_nganh_1 != None)) & ((check_nganh_2 == True) | (prev1_nganh_2 == None)):
                if current_nganh_list[0] == prev1_nganh_1:
                    final_nganh_list = [current_nganh_list[1], prev1_nganh_1]
                else:
                    final_nganh_list = [current_nganh_list[0], prev1_nganh_1]
            elif ((check_nganh_2 == False) & (prev1_nganh_2 != None)) & ((check_nganh_1 == True) | (prev1_nganh_1 == None)):
                if current_nganh_list[0] == prev1_nganh_2:
                    final_nganh_list = [prev1_nganh_2, current_nganh_list[1]]
                else:
                    final_nganh_list = [prev1_nganh_2, current_nganh_list[0]]
            else:
                final_nganh_list = current_nganh_list

        elif len(current_nganh_list) == 1:
            if ((check_nganh_1 == False) & (prev1_nganh_1 != None)) & ((check_nganh_2 == False) & (prev1_nganh_2 != None)):
                final_nganh_list = prev1_nganh_list
            elif ((check_nganh_1 == False) & (prev1_nganh_1 != None)) & ((check_nganh_2 == True) | (prev1_nganh_2 == None)):
                if current_nganh_list[0] == prev1_nganh_1:
                    final_nganh_list = [prev1_nganh_1]
                else:
                    final_nganh_list = [current_nganh_list[0], prev1_nganh_1]
            elif ((check_nganh_2 == False) & (prev1_nganh_2 != None)) & ((check_nganh_1 == True) | (prev1_nganh_1 == None)):
                if current_nganh_list[0] == prev1_nganh_2:
                    final_nganh_list = [prev1_nganh_2]
                else:
                    final_nganh_list = [prev1_nganh_2, current_nganh_list[0]]
            else:
                final_nganh_list = current_nganh_list

        elif len(current_nganh_list) == 0:
            if ((check_nganh_1 == False) & (prev1_nganh_1 != None)) & ((check_nganh_2 == False) & (prev1_nganh_2 != None)):
                final_nganh_list = prev1_nganh_list
            elif ((check_nganh_1 == False) & (prev1_nganh_1 != None)) & ((check_nganh_2 == True) | (prev1_nganh_2 == None)):
                final_nganh_list = [prev1_nganh_1]
            elif ((check_nganh_2 == False) & (prev1_nganh_2 != None)) & ((check_nganh_1 == True) | (prev1_nganh_1 == None)):
                final_nganh_list = [prev1_nganh_2]
            else:
                final_nganh_list = current_nganh_list

        #Tiến hành gán lại giá trị
        df.loc[i, column_list] = 0
        if len (final_nganh_list) == 2:
            df.at[i, final_nganh_list[0]] = 0.5
            df.at[i, final_nganh_list[1]] = 0.5
        elif len (final_nganh_list) == 1:
            df.at[i, final_nganh_list[0]] = 0.5

    return df[column_list]

In [129]:
def adjust_group_portion_T3(df, column_list):
    for i in range(len(df) - 4, -1, -1):

        prev3_nganh_series = df[column_list].iloc[i+3]
        prev3_nganh_list = prev3_nganh_series[prev3_nganh_series > 0].index.tolist()

        prev2_nganh_series = df[column_list].iloc[i+2]
        prev2_nganh_list = prev2_nganh_series[prev2_nganh_series > 0].index.tolist()

        prev1_nganh_series = df[column_list].iloc[i+1]
        prev1_nganh_list = prev1_nganh_series[prev1_nganh_series > 0].index.tolist()

        try: prev1_nganh_1 = prev1_nganh_list[0]
        except: prev1_nganh_1 = None
        try: prev1_nganh_2 = prev1_nganh_list[1]
        except: prev1_nganh_2 = None

        #Kiểm tra xem ngành 1 đã bán được chưa
        if (prev1_nganh_1 in prev2_nganh_list) & (prev1_nganh_1 in prev3_nganh_list):
            check_nganh_1 = True
        else: check_nganh_1 = False

        #Kiểm tra xem ngành 2 đã bán được chưa
        if (prev1_nganh_2 in prev2_nganh_list) & (prev1_nganh_2 in prev3_nganh_list):
            check_nganh_2 = True
        else: check_nganh_2 = False

        #Tạo danh sách ngành hiện tại khi chưa hiệu chỉnh
        current_nganh_series = df[column_list].iloc[i]
        current_nganh_list = current_nganh_series[current_nganh_series > 0].index.tolist()

        #Hiệu chỉnh danh sách ngành cho đúng T3
        if len(current_nganh_list) == 2:
            if ((check_nganh_1 == False) & (prev1_nganh_1 != None)) & ((check_nganh_2 == False) & (prev1_nganh_2 != None)):
                final_nganh_list = prev1_nganh_list
            elif ((check_nganh_1 == False) & (prev1_nganh_1 != None)) & ((check_nganh_2 == True) | (prev1_nganh_2 == None)):
                if current_nganh_list[0] == prev1_nganh_1:
                    final_nganh_list = [current_nganh_list[1], prev1_nganh_1]
                else:
                    final_nganh_list = [current_nganh_list[0], prev1_nganh_1]
            elif ((check_nganh_2 == False) & (prev1_nganh_2 != None)) & ((check_nganh_1 == True) | (prev1_nganh_1 == None)):
                if current_nganh_list[0] == prev1_nganh_2:
                    final_nganh_list = [prev1_nganh_2, current_nganh_list[1]]
                else:
                    final_nganh_list = [prev1_nganh_2, current_nganh_list[0]]
            else:
                final_nganh_list = current_nganh_list

        elif len(current_nganh_list) == 1:
            if ((check_nganh_1 == False) & (prev1_nganh_1 != None)) & ((check_nganh_2 == False) & (prev1_nganh_2 != None)):
                final_nganh_list = prev1_nganh_list
            elif ((check_nganh_1 == False) & (prev1_nganh_1 != None)) & ((check_nganh_2 == True) | (prev1_nganh_2 == None)):
                if current_nganh_list[0] == prev1_nganh_1:
                    final_nganh_list = [prev1_nganh_1]
                else:
                    final_nganh_list = [current_nganh_list[0], prev1_nganh_1]
            elif ((check_nganh_2 == False) & (prev1_nganh_2 != None)) & ((check_nganh_1 == True) | (prev1_nganh_1 == None)):
                if current_nganh_list[0] == prev1_nganh_2:
                    final_nganh_list = [prev1_nganh_2]
                else:
                    final_nganh_list = [prev1_nganh_2, current_nganh_list[0]]
            else:
                final_nganh_list = current_nganh_list

        elif len(current_nganh_list) == 0:
            if ((check_nganh_1 == False) & (prev1_nganh_1 != None)) & ((check_nganh_2 == False) & (prev1_nganh_2 != None)):
                final_nganh_list = prev1_nganh_list
            elif ((check_nganh_1 == False) & (prev1_nganh_1 != None)) & ((check_nganh_2 == True) | (prev1_nganh_2 == None)):
                final_nganh_list = [prev1_nganh_1]
            elif ((check_nganh_2 == False) & (prev1_nganh_2 != None)) & ((check_nganh_1 == True) | (prev1_nganh_1 == None)):
                final_nganh_list = [prev1_nganh_2]
            else:
                final_nganh_list = current_nganh_list

        #Tiến hành gán lại giá trị
        df.loc[i, column_list] = 0
        if len (final_nganh_list) == 2:
            df.at[i, final_nganh_list[0]] = 0.5
            df.at[i, final_nganh_list[1]] = 0.5
        elif len (final_nganh_list) == 1:
            df.at[i, final_nganh_list[0]] = 0.5

    return df[column_list]

In [130]:
def count_phase(df, column_name):
    # Tạo list và giá trị đếm
    reverse_count = []
    count = 0
    # Duyệt qua các giá trị của cột từ dưới lên
    for value in reversed(df[column_name]):
        if value != 0:
            count += 1
        else:
            count = 0
        reverse_count.append(count)
    # Đảo ngược lại danh sách kết quả để đúng thứ tự ban đầu
    reverse_count.reverse()
    return reverse_count

def pick_nganh(initial_df, industry_dict, nganh_list):
    df = initial_df[[f'{item}_portion' for item in nganh_list]]

    for nganh in nganh_list:
        df[f'{nganh}_count'] = count_phase(df, f'{nganh}_portion')

    df['top_nganh'] = None

    for i in range(len(df) - 1, -1, -1):
        top_nganh_portion = df[[f'{item}_portion' for item in nganh_list]].iloc[i].sort_values(ascending=False)
        top_nganh_portion_1 = df[[f'{item}_portion' for item in nganh_list]].iloc[i].sort_values(ascending=False).index[0].replace('_portion', '')
        top_nganh_portion_2 = df[[f'{item}_portion' for item in nganh_list]].iloc[i].sort_values(ascending=False).index[1].replace('_portion', '')

        count_nganh_portion = top_nganh_portion[(top_nganh_portion != 0) & (top_nganh_portion.notna())].count()

        #Tìm ra ngành có count phase < 3 và portion lớn nhất
        temp_series = df.iloc[i].drop('top_nganh')
        portion_series = temp_series[temp_series.index.str.endswith('_portion')].rename(lambda x: x.replace('_portion', ''))
        count_series = temp_series[temp_series.index.str.endswith('_count')].rename(lambda x: x.replace('_count', ''))
        temp_df = pd.DataFrame({'portion': portion_series, 'count': count_series}).sort_values(['count', 'portion'], ascending=[True, False])
        temp_df = temp_df[(temp_df['count'] <= 3) & (temp_df['count'] != 0) & (temp_df['portion'] != 0)]

        if len(temp_df) > 0:
            top_nganh_phase = temp_df.index[0]
        else:
            top_nganh_phase = None

        df.at[i, 'top_nganh_phase'] = top_nganh_phase
        
        if i == (len(df) - 1):
            df.at[i, 'top_nganh'] = []
        elif i == (len(df) - 2):
            df.at[i, 'top_nganh'] = [top_nganh_portion_1,top_nganh_portion_2]
        else:
            prev_list_nganh = df['top_nganh'].iloc[i+1]
            if len(prev_list_nganh) == 2:
                if (df[f'{prev_list_nganh[0]}_portion'].iloc[i] == 0) & (df[f'{prev_list_nganh[1]}_portion'].iloc[i] != 0):
                    if count_nganh_portion == 0:
                        df.at[i, 'top_nganh'] = []
                    elif count_nganh_portion == 1:
                        df.at[i, 'top_nganh'] = [prev_list_nganh[1]]
                    else:
                        if (top_nganh_phase != prev_list_nganh[1]) & (top_nganh_phase != None):
                            df.at[i, 'top_nganh'] = [top_nganh_phase, prev_list_nganh[1]]
                        else:
                            if top_nganh_portion_1 == prev_list_nganh[1]:
                                df.at[i, 'top_nganh'] = [top_nganh_portion_2, prev_list_nganh[1]]
                            else:
                                df.at[i, 'top_nganh'] = [top_nganh_portion_1, prev_list_nganh[1]]


                elif (df[f'{prev_list_nganh[0]}_portion'].iloc[i] != 0) & (df[f'{prev_list_nganh[1]}_portion'].iloc[i] == 0):
                    if count_nganh_portion == 0:
                        df.at[i, 'top_nganh'] = []
                    elif count_nganh_portion == 1:
                        df.at[i, 'top_nganh'] = [prev_list_nganh[0]]
                    else:
                        if (top_nganh_phase != prev_list_nganh[0]) & (top_nganh_phase != None):
                            df.at[i, 'top_nganh'] = [top_nganh_phase, prev_list_nganh[0]]
                        else:
                            if top_nganh_portion_1 == prev_list_nganh[0]:
                                df.at[i, 'top_nganh'] = [top_nganh_portion_2, prev_list_nganh[0]]
                            else:
                                df.at[i, 'top_nganh'] = [top_nganh_portion_1, prev_list_nganh[0]]

                elif (df[f'{prev_list_nganh[0]}_portion'].iloc[i] == 0) & (df[f'{prev_list_nganh[1]}_portion'].iloc[i] == 0):
                    if count_nganh_portion == 0:
                        df.at[i, 'top_nganh'] = []
                    elif count_nganh_portion == 1:
                        if top_nganh_phase != None:
                            df.at[i, 'top_nganh'] = [top_nganh_phase]
                        else:                   
                            df.at[i, 'top_nganh'] = [top_nganh_portion_1]
                    else:
                        if (top_nganh_phase != top_nganh_portion_1) & (top_nganh_phase != None):
                            df.at[i, 'top_nganh'] = [top_nganh_phase, top_nganh_portion_1]
                        else:
                            df.at[i, 'top_nganh'] = [top_nganh_portion_1,top_nganh_portion_2]
                else:
                    df.at[i, 'top_nganh'] = prev_list_nganh

            elif len(prev_list_nganh) == 1:
                if df[f'{prev_list_nganh[0]}_portion'].iloc[i] == 0:
                    if count_nganh_portion == 0:
                        df.at[i, 'top_nganh'] = []
                    elif count_nganh_portion == 1:
                        if top_nganh_phase != None:
                            df.at[i, 'top_nganh'] = [top_nganh_phase]
                        else:                   
                            df.at[i, 'top_nganh'] = [top_nganh_portion_1]
                    else:
                        if (top_nganh_phase != top_nganh_portion_1) & (top_nganh_phase != None):
                            df.at[i, 'top_nganh'] = [top_nganh_phase, top_nganh_portion_1]
                        else:
                            df.at[i, 'top_nganh'] = [top_nganh_portion_1,top_nganh_portion_2]
                else:
                    if count_nganh_portion == 0:
                        df.at[i, 'top_nganh'] = []
                    elif count_nganh_portion == 1:
                        df.at[i, 'top_nganh'] = [prev_list_nganh[0]]
                    else:
                        if (top_nganh_phase != top_nganh_portion_1) & (top_nganh_phase != None):
                            df.at[i, 'top_nganh'] = [top_nganh_phase, prev_list_nganh[0]]
                        else:
                            df.at[i, 'top_nganh'] = [top_nganh_portion_1,prev_list_nganh[0]]
            
            elif len(prev_list_nganh) == 0:
                    if count_nganh_portion == 0:
                        df.at[i, 'top_nganh'] = []
                    elif count_nganh_portion == 1:
                        if top_nganh_phase != None:
                            df.at[i, 'top_nganh'] = [top_nganh_phase]
                        else:                   
                            df.at[i, 'top_nganh'] = [top_nganh_portion_1]
                    else:
                        if (top_nganh_phase != top_nganh_portion_1) & (top_nganh_phase != None):
                            df.at[i, 'top_nganh'] = [top_nganh_phase, top_nganh_portion_1]
                        else:
                            df.at[i, 'top_nganh'] = [top_nganh_portion_1,top_nganh_portion_2]

    return df['top_nganh']

In [131]:
def phan_bo_von_industry_raw(nganh_list):
    phan_bo_von_industry_raw = pd.DataFrame()
    phan_bo_von_industry_raw['date'] = date_series['date']

    phan_bo_von_industry_raw[[item + '+' for item in nganh_list]] = full_group_score_df_positive[nganh_list][::-1].rolling(window=5, min_periods=1).mean()[::-1]
    phan_bo_von_industry_raw[[item + '-' for item in nganh_list]] = full_group_score_df_negative[nganh_list][::-1].rolling(window=5, min_periods=1).mean().abs()[::-1]

    for column in nganh_list:
        phan_bo_von_industry_raw[f'{column}_portion'] = ((phan_bo_von_industry_raw[f'{column}+'] - phan_bo_von_industry_raw[f'{column}-']) / 
                                                        (phan_bo_von_industry_raw[f'{column}+'] + phan_bo_von_industry_raw[f'{column}-'])).clip(lower=0)

    phan_bo_von_industry_raw = phan_bo_von_industry_raw.fillna(0)
    phan_bo_von_industry_raw = phan_bo_von_industry_raw[['date'] + [item + '_portion' for item in nganh_list]]

    return phan_bo_von_industry_raw

In [132]:
def phan_bo_von_industry_dict(phan_bo_von_industry_raw, nganh_list, nganh_deleted):
    phan_bo_von_industry_dict = {}
    for nganh in nganh_list:
        phan_bo_von_industry_dict[nganh] = pd.DataFrame()
        phan_bo_von_industry_dict[nganh]['index'] = group_price_index_df[nganh]
        phan_bo_von_industry_dict[nganh]['total_portion'] = phan_bo_von_final['final_portion']
        phan_bo_von_industry_dict[nganh][['date','portion_raw']] = phan_bo_von_industry_raw[['date', nganh + '_portion']]

        temp_ms = calculate_group_ms(full_market_ms[full_market_ms['name']==nganh][['date','trend_5p','trend_20p','trend_60p']]).sort_values('date', ascending=False).reset_index(drop=True)
        phan_bo_von_industry_dict[nganh] = phan_bo_von_industry_dict[nganh].merge(temp_ms[['date','up_check','down_check']], on='date', how='left')

        #Tạo các giai đoạn mua và bán
        phan_bo_von_industry_dict[nganh]['portion_phase'] = calculate_portion_phase(phan_bo_von_industry_dict[nganh])
        phan_bo_von_industry_dict[nganh]['final_portion'] = calculate_final_portion_nhom(phan_bo_von_industry_dict[nganh])

        if nganh in nganh_deleted:
            phan_bo_von_industry_dict[nganh]['final_portion'] = 0

    return phan_bo_von_industry_dict

In [133]:
def phan_bo_von_industry_final(phan_bo_von_industry_dict, phan_bo_von_industry_raw, nganh_list, column):
    phan_bo_von_industry_final = pd.DataFrame()
    phan_bo_von_industry_final['date'] = phan_bo_von_industry_raw['date']
    phan_bo_von_industry_final['total_portion'] = phan_bo_von_final[column]

    for nganh in nganh_list:
        phan_bo_von_industry_final[nganh + '_portion'] = phan_bo_von_industry_dict[nganh]['final_portion']
        #Điều chỉnh để nếu tỉ trọng tổng bằng 0 thì ngành cũng bằng 0
        phan_bo_von_industry_final[nganh + '_portion'] = phan_bo_von_industry_final.apply(lambda x: 0 if x['total_portion'] == 0 else x[nganh + '_portion'], axis=1)

    # # Tạo cột chọn các ngành được pick
    phan_bo_von_industry_final['top_nganh'] = pick_nganh(phan_bo_von_industry_final, phan_bo_von_industry_dict, nganh_list)

    #Xoá các tỉ trọng của những ngành không được chọn về 0 và ngành được chọn là 0.5
    for i in range(len(phan_bo_von_industry_final)):
        for nganh in [f'{item}_portion' for item in nganh_list]:
            if nganh.replace('_portion', '') not in phan_bo_von_industry_final['top_nganh'].iloc[i]:
                phan_bo_von_industry_final.at[i, nganh] = 0
            else:
                phan_bo_von_industry_final.at[i, nganh] = 0.5

    #Điều chỉnh để tỉ trọng ko thay đổi 2 phiên liên tiếp
    phan_bo_von_industry_final[[item + '_portion' for item in nganh_list]] = adjust_group_portion_T3(phan_bo_von_industry_final, [item + '_portion' for item in nganh_list])

    #Tính toán sự thay đổi index
    for nganh in nganh_list:
        phan_bo_von_industry_final[nganh + '_index'] = group_price_index_df[nganh]
    for nganh in nganh_list:
        phan_bo_von_industry_final[nganh + '_index_change'] = phan_bo_von_industry_final[nganh + '_index'][::-1].pct_change()[::-1]
    phan_bo_von_industry_final = phan_bo_von_industry_final.fillna(0)

    #Tính thay đổi của index tổng của nhóm hiệu suất theo tỉ trọng của từng nhóm ngành
    phan_bo_von_industry_final['total_index_change'] = phan_bo_von_industry_final.apply(lambda row: calculate_group_total_change(row, nganh_list), axis=1)

    #Lọc ra các cột cần thiết
    filtered_columns = [col for col in phan_bo_von_industry_final.columns if col.endswith('_portion') or col.endswith('_index_change') or col == 'date']
    phan_bo_von_industry_final = phan_bo_von_industry_final[filtered_columns]

    return phan_bo_von_industry_final

In [134]:
nganh_hsA_list = ['ban_le','bds','chung_khoan','tai_chinh','thep','vlxd','xd']
nganh_hsA_deleted = ['vlxd','tai_chinh','xd']
phan_bo_von_hsA_raw = phan_bo_von_industry_raw(nganh_hsA_list)
phan_bo_von_hsA_dict = phan_bo_von_industry_dict(phan_bo_von_hsA_raw, nganh_hsA_list, nganh_hsA_deleted)
phan_bo_von_hsA_final = phan_bo_von_industry_final(phan_bo_von_hsA_dict, phan_bo_von_hsA_raw, nganh_hsA_list, 'A_portion')

nganh_hsB_list = ['det_may','cong_nghiep','hoa_chat','dau_khi','thuy_san','khoang_san'] 
nganh_hsB_deleted = ['cong_nghiep','khoang_san']
phan_bo_von_hsB_raw = phan_bo_von_industry_raw(nganh_hsB_list)
phan_bo_von_hsB_dict = phan_bo_von_industry_dict(phan_bo_von_hsB_raw, nganh_hsB_list, nganh_hsB_deleted)
phan_bo_von_hsB_final = phan_bo_von_industry_final(phan_bo_von_hsB_dict, phan_bo_von_hsB_raw, nganh_hsB_list, 'B_portion')

nganh_hsC_list = ['bds_kcn','thuc_pham','van_tai','cong_nghe','htd','ngan_hang']
nganh_hsC_deleted = ['htd','thuc_pham']
phan_bo_von_hsC_raw = phan_bo_von_industry_raw(nganh_hsC_list)
phan_bo_von_hsC_dict = phan_bo_von_industry_dict(phan_bo_von_hsC_raw, nganh_hsC_list, nganh_hsC_deleted)
phan_bo_von_hsC_final = phan_bo_von_industry_final(phan_bo_von_hsC_dict, phan_bo_von_hsC_raw, nganh_hsC_list, 'C_portion')

#### Tín hiệu cổ phiếu

##### Chuẩn bị dữ liệu

In [135]:
hs_stock_backtest = phan_bo_von_final[['date','A_portion','B_portion','C_portion','D_portion']]
hs_stock_backtest.columns = ['date'] + industry_perform_list

hsA_stock_backtest = phan_bo_von_hsA_final[['date'] + [f'{item}_portion' for item in nganh_hsA_list]]
hsA_stock_backtest.columns = ['date'] + nganh_hsA_list

hsB_stock_backtest = phan_bo_von_hsB_final[['date'] + [f'{item}_portion' for item in nganh_hsB_list]]
hsB_stock_backtest.columns = ['date'] + nganh_hsB_list

hsC_stock_backtest = phan_bo_von_hsC_final[['date'] + [f'{item}_portion' for item in nganh_hsC_list]]
hsC_stock_backtest.columns = ['date'] + nganh_hsC_list

##### Các hàm tính toán

In [136]:
#Thêm cột tên period và số lượng cổ phiếu từng thời kì
def assign_period(x):
    for key, value in period_map_dict.items():
        if (x >= pd.Timestamp(value[0])) & (x <= pd.Timestamp(value[1])):
            return key

#Thêm cột stock để thể hiện thông tin các cổ phiếu mua bán
def calculate_stock_list(df):
    df['stock'] = None
    for i in range(len(df) - 1, -1, -1):

        #Tạo tỉ trọng các nhóm
        a_portion = df['A'].iloc[i]
        b_portion = df['B'].iloc[i]
        c_portion = df['C'].iloc[i]

        #Tạo danh sách các nhóm trong nhóm lớn
        a_list = [key for key, value in a_portion.items() if value > 0]
        b_list = [key for key, value in b_portion.items() if value > 0]
        c_list = [key for key, value in c_portion.items() if value > 0]

        #Tạo danh sách cổ phiếu theo ngày
        temp_stock_classification_df = period_stock_classification_dict[df['period'].iloc[i]]
        stock_list = temp_stock_classification_df[temp_stock_classification_df['industry_name'].isin(a_list + b_list + c_list)]['stock'].tolist()

        #Lọc cổ phiếu dựa trên điều kiện thanh khoản cao hơn
        stock_list = list(set(stock_list) & set(period_signal_stock_dict[df['period'].iloc[i]].keys()))
        
        if i == len(df) - 1:
            df.at[i, 'stock'] = stock_list
        else:
            if df[['hs', 'A', 'B', 'C']].iloc[i].equals(df[['hs', 'A', 'B', 'C']].iloc[i+1]):
                df.at[i, 'stock'] = None
            else:
                df.at[i, 'stock'] = stock_list

    return df['stock']

##### Bộ lọc cổ phiếu

In [137]:
period_signal_stock_dict = copy.deepcopy(period_stock_dict)
for period in period_stock_df.columns[1:]:
    for stock, df in period_signal_stock_dict[period].items():
        df['ma20_volume'] = df['volume'][::-1].rolling(window=20, min_periods=1).mean()[::-1]
        df['value'] = df['volume']*df['close']
        df['mean_value'] = df['value'].mean()
        df['ma20_value'] = df['value'][::-1].rolling(window=20, min_periods=1).mean()[::-1]
        period_signal_stock_dict[period][stock] = df

    period_signal_stock_dict[period] = {k: v for k, v in period_signal_stock_dict[period].items() if (v['volume'].mean() >= 100000) & (v['ma20_volume'].min() >= 100000)&
                                                                                                     (v['value'].mean() >= 2000000) & (v['ma20_value'].min() >= 2000000)}

In [138]:
#Bảng dòng tiền T5 và bién động trong 20 phiên của CP
stock_score_change_df = date_series.copy()
stock_t5_score_df = date_series.copy()

for stock in full_stock_score_dict.keys():
    temp_df = date_series.copy()
    temp_df['t0_score'] = full_stock_score_dict[stock]['t0_score']
    temp_df['sum_score_change'] = temp_df['t0_score'][::-1].rolling(window=20, min_periods=1).sum()[::-1]
    temp_df['root_score_change'] = pd.concat([temp_df['t0_score'].iloc[:-1].shift(-19).ffill(), pd.Series([0])], ignore_index=True)
    temp_df['score_change'] = (temp_df['sum_score_change'] - temp_df['root_score_change'])/100

    stock_score_change_df[stock] = temp_df['score_change']
    stock_t5_score_df[stock] = full_stock_score_dict[stock]['t5_score']

In [139]:
#Bảng dòng tiền T5 và bién động trong 20 phiên của nhóm CP
group_score_change_df = date_series.copy()
group_t5_score_df = date_series.copy()

for group in group_stock_key_list:
    temp_df = date_series.copy()
    temp_df['t0_score'] = full_group_score_df_net[group]
    temp_df['sum_score_change'] = temp_df['t0_score'][::-1].rolling(window=20, min_periods=1).sum()[::-1]
    temp_df['root_score_change'] = pd.concat([temp_df['t0_score'].iloc[:-1].shift(-19).ffill(), pd.Series([0])], ignore_index=True)
    temp_df['score_change'] = (temp_df['sum_score_change'] - temp_df['root_score_change'])/100

    group_score_change_df[group] = temp_df['score_change']
    group_t5_score_df[group] = full_group_score_df_net[group][::-1].rolling(window=5, min_periods=1).mean()[::-1]

##### TÍnh toán hiệu suất

In [140]:
#Bảng thông tin mua bán cổ phiếu từng ngày
stock_backtest_df = phan_bo_von_final[['date','final_portion']]
stock_backtest_df['period'] = phan_bo_von_final['date'].apply(assign_period)

stock_backtest_df['hs'] = hs_stock_backtest.apply(lambda row: {col: round(row[col],1) for col in industry_perform_list}, axis=1)

stock_backtest_df['A'] = hsA_stock_backtest.apply(lambda row: {col: round(row[col],1) for col in nganh_hsA_list}, axis=1)
stock_backtest_df['B'] = hsB_stock_backtest.apply(lambda row: {col: round(row[col],1) for col in nganh_hsB_list}, axis=1)
stock_backtest_df['C'] = hsC_stock_backtest.apply(lambda row: {col: round(row[col],1) for col in nganh_hsC_list}, axis=1)

stock_backtest_df['stock'] = calculate_stock_list(stock_backtest_df).bfill()

# #Lọc lại danh sách cổ phiếu theo ngày chỉ chọn ra 2 cổ top
# def get_top_stock(stock_list, date):
#     nganh_stock_dict = {}
#     for stock in stock_list:
#         nganh = full_stock_classification_df[full_stock_classification_df['stock'] == stock]['industry_name'].item()
#         if nganh not in nganh_stock_dict.keys():
#             nganh_stock_dict[nganh] = [stock]
#         else:
#             nganh_stock_dict[nganh].append(stock)
    
#     for nganh, nganh_stock_list in nganh_stock_dict.items():
#         nganh_stock_dict[nganh] = stock_t5_score_df[stock_t5_score_df['date'] == date][nganh_stock_list].iloc[0].sort_values(ascending=False).index[:2].tolist()
    
#         # t5_score_stock_dict = stock_t5_score_df[stock_t5_score_df['date'] == date][nganh_stock_list].iloc[0].to_dict()
#         # score_change_stock_dict = stock_score_change_df[stock_score_change_df['date'] == date][nganh_stock_list].iloc[0].to_dict()

#         # nganh_t5_score = group_t5_score_df[group_t5_score_df['date'] == date][nganh].item()
#         # nganh_score_change = group_score_change_df[group_score_change_df['date'] == date][nganh].item()

#         # # for stock in nganh_stock_list:
#         # #     if t5_score_stock_dict[stock] < nganh_t5_score:
#         # #         t5_score_stock_dict.pop(stock)
#         #     # if score_change_stock_dict[stock] < nganh_score_change:
#         #     #     score_change_stock_dict.pop(stock)

#         # filtered_stock_list = list(set(t5_score_stock_dict.keys()) & set(score_change_stock_dict.keys()))
#         # # filtered_stock_list = stock_t5_score_df[stock_t5_score_df['date'] == date][filtered_stock_list].iloc[0].sort_values(ascending=False).index.tolist()
#         # nganh_stock_dict[nganh] = filtered_stock_list
        
#     #Ghép nối tất cả cổ phiếu vào ngành
#     final_stock_list = []
#     for nganh_stock_list in nganh_stock_dict.values():
#         final_stock_list.extend(nganh_stock_list)

#     return final_stock_list

In [141]:
#Tạo bảng tín hiệu mua bán cổ phiếu
stock_signal_df = date_series.copy()
for stock in full_stock_dict.keys():
    stock_signal_df[stock] = 0
    for i in range(len(stock_signal_df)):
        if stock in stock_backtest_df['stock'].iloc[i]:
            stock_signal_df.at[i, stock] = 1

#Bảng thông tin biến động cổ phiếu từng ngày
stock_price_change_df = date_series.copy()
for stock, stock_df in full_stock_dict.items():
    stock_price_change_df[stock] = 0
    for i in range(len(stock_price_change_df) - 3, -1, -1):
        if len(stock_df) >= i+2:
            if (stock_signal_df[stock].iloc[i+2] == 0) & (stock_signal_df[stock].iloc[i+1] == 1):
                stock_price_change_df.at[i, stock] = (stock_df['close'].iloc[i] - stock_df['open'].iloc[i]) / stock_df['open'].iloc[i]

            elif (stock_signal_df[stock].iloc[i+2] == 1) & (stock_signal_df[stock].iloc[i+1] == 0):
                stock_price_change_df.at[i, stock] = (stock_df['open'].iloc[i] - stock_df['close'].iloc[i+1]) / stock_df['close'].iloc[i+1]

            elif (stock_signal_df[stock].iloc[i+2] == 1) & (stock_signal_df[stock].iloc[i+1] == 1):
                stock_price_change_df.at[i, stock] = (stock_df['close'].iloc[i] - stock_df['close'].iloc[i+1]) / stock_df['close'].iloc[i+1]

In [142]:
#Tính toán biến động dựa theo số lượng cổ phiếu mua bán
stock_backtest_df['pct_stock_index'] = 0
for i in range(len(stock_backtest_df)-1, -1, -1):
    
    pct_list = []

    #Cổ phiếu mở mua mới hoặc đang nắm giữ
    stock_list_1 = stock_backtest_df['stock'].iloc[i]

    #Cổ phiếu đang nắm giữ hoặc chưa kịp bán do ko đủ T
    stock_series = stock_price_change_df.iloc[i, 1:]
    stock_list_2 = stock_series[stock_series != 0].index.tolist()

    #Tổng danh sách cổ phiếu
    stock_list = list(set(stock_list_1) | set(stock_list_2))

    #Tạo dict chứa số lượng cổ phiếu theo từng phân loại
    temp_df = full_stock_classification_df[full_stock_classification_df['stock'].isin(stock_list)]
    temp_df = temp_df.groupby('industry_name')['stock'].count().reset_index()
    stock_count_dict = { row['industry_name']: row['stock'] for _, row in temp_df.iterrows() }

    for stock in stock_list:
        stock_industry_name = full_stock_classification_df[full_stock_classification_df['stock']==stock]['industry_name'].item()
        stock_industry_perform = full_stock_classification_df[full_stock_classification_df['stock']==stock]['industry_perform'].item()
        count = stock_count_dict[stock_industry_name]

        #Nếu cổ phiếu thuộc danh sách đang nắm giữ
        if stock in stock_backtest_df['stock'].iloc[i]:
            industry_name_portion = stock_backtest_df.iloc[i][stock_industry_perform][stock_industry_name]
            industry_perform_portion = stock_backtest_df.iloc[i]['hs'][stock_industry_perform]

        #Nếu cổ phiếu ko thuộc danh sách đang nắm giữ thì tìm nó ở các phiên trước
        else:
            for j in range(1,7):
                if stock in stock_backtest_df['stock'].iloc[i+j]:
                    industry_name_portion = stock_backtest_df.iloc[i+j][stock_industry_perform][stock_industry_name]
                    industry_perform_portion = stock_backtest_df.iloc[i+j]['hs'][stock_industry_perform]

                    if (industry_name_portion != 0) & (industry_perform_portion != 0):
                     break

        pct_value = (stock_price_change_df[stock].iloc[i] * industry_name_portion * industry_perform_portion) / count
        pct_list.append(pct_value)

    stock_backtest_df.at[i, 'pct_stock_index'] = sum(pct_list)

#### Backtest hiệu năng

In [143]:
def calculate_profit(df, money_column, pct_column):
    df_copy = df.copy()
    for i in range(len(df_copy) - 2, -1, -1):
        if df_copy['final_portion'].iloc[i+1] == 1:
            df_copy.loc[i, money_column] = df_copy[money_column].iloc[i+1] + df_copy[money_column].iloc[i+1]*df_copy[pct_column].iloc[i]
        else:
            df_copy.loc[i, money_column] = df_copy.loc[i + 1, money_column]
    
    return df_copy

def calculate_stock_profit(df, money_column, pct_column):
    df_copy = df.copy()
    for i in range(len(df_copy) - 2, -1, -1):
        df_copy.loc[i, money_column] = df_copy[money_column].iloc[i+1] + df_copy[money_column].iloc[i+1]*df_copy[pct_column].iloc[i]
    return df_copy

In [144]:
phan_bo_von_backtest = copy.deepcopy(phan_bo_von_final)

# phan_bo_von_backtest = phan_bo_von_final[(phan_bo_von_final['date'] >= '2024-01-01') & (phan_bo_von_final['date'] < '2025-01-01')]
# phan_bo_von_backtest = phan_bo_von_final[phan_bo_von_final['date'] >= '2022-11-22']
# phan_bo_von_backtest = phan_bo_von_final[phan_bo_von_final['date'] >= '2023-10-30']

#total_index_change được tính theo tỉ trọng của từng ngành vào nhóm vốn hoá, sau đó ở bước này ta lại nhân với tỉ trọng từng nhóm vốn hoá
phan_bo_von_backtest['pct_nganh_index'] = phan_bo_von_hsA_final['total_index_change'] * phan_bo_von_backtest['A_portion']\
                                        + phan_bo_von_hsB_final['total_index_change'] * phan_bo_von_backtest['B_portion']\
                                        + phan_bo_von_hsC_final['total_index_change'] * phan_bo_von_backtest['C_portion']

#Lấy cột VNINDEX để tính % thay đổi của VNINDEX từng phiên
phan_bo_von_backtest['vnindex'] = group_price_index_df['VNINDEX']
phan_bo_von_backtest['pct_vnindex'] = phan_bo_von_backtest['vnindex'][::-1].pct_change()[::-1]

#Lấy cột all_stock_index để tính % thay đổi của all_stock_index từng phiên
phan_bo_von_backtest['t2m_index'] = group_price_index_df['all_stock']
phan_bo_von_backtest['pct_t2m_index'] = phan_bo_von_backtest['t2m_index'][::-1].pct_change()[::-1]

#Lấy cột pct_stock_index
phan_bo_von_backtest['pct_stock_index'] = stock_backtest_df['pct_stock_index']

# Khởi tạo giá trị cho cột money ban đầu
initial_money = phan_bo_von_backtest['vnindex'].iloc[-1]
phan_bo_von_backtest['money_vnindex'] = 0
phan_bo_von_backtest['money_vnindex'].iloc[-1] = initial_money

phan_bo_von_backtest['money_t2m_index'] = phan_bo_von_backtest['money_vnindex']
phan_bo_von_backtest['money_nganh_index'] = phan_bo_von_backtest['money_vnindex']
phan_bo_von_backtest['money_stock_index'] = phan_bo_von_backtest['money_vnindex']

# Áp dụng hàm tính toán
phan_bo_von_backtest = calculate_profit(phan_bo_von_backtest, 'money_vnindex', 'pct_vnindex')
phan_bo_von_backtest = calculate_profit(phan_bo_von_backtest, 'money_t2m_index', 'pct_t2m_index')
phan_bo_von_backtest = calculate_profit(phan_bo_von_backtest, 'money_nganh_index', 'pct_nganh_index')
phan_bo_von_backtest = calculate_stock_profit(phan_bo_von_backtest, 'money_stock_index', 'pct_stock_index')

phan_bo_von_backtest = phan_bo_von_backtest.fillna(0)

with pd.ExcelWriter('test_data.xlsx', engine='openpyxl') as writer:
    full_market_ms[full_market_ms['name']=='all_stock'].to_excel(writer, sheet_name='full_market_ms', index=False)
    phan_bo_von_backtest.to_excel(writer, sheet_name='phan_bo_von_backtest', index=False)
    phan_bo_von_hsA_final.to_excel(writer, sheet_name='phan_bo_von_hsA_final', index=False)
    phan_bo_von_hsB_final.to_excel(writer, sheet_name='phan_bo_von_hsB_final', index=False)
    phan_bo_von_hsC_final.to_excel(writer, sheet_name='phan_bo_von_hsC_final', index=False)

# with pd.ExcelWriter(f"history/{datetime.now().date().strftime('%d_%m_%Y')}.xlsx", engine='openpyxl') as writer:
#     phan_bo_von_backtest.to_excel(writer, sheet_name='phan_bo_von_backtest', index=False)
#     full_market_ms[full_market_ms['name']=='all_stock'].to_excel(writer, sheet_name='full_market_ms', index=False)
#     phan_bo_von_hsA_final.to_excel(writer, sheet_name='phan_bo_von_hsA_final', index=False)
#     phan_bo_von_hsB_final.to_excel(writer, sheet_name='phan_bo_von_hsB_final', index=False)
#     phan_bo_von_hsC_final.to_excel(writer, sheet_name='phan_bo_von_hsC_final', index=False)